In [ ]:
#Code compiled for Shin et al. 2024
#Last amended 9/29/24 by Maddie Urbanek
#Components:
    #Terminal
        #Rabies barcode calling
        #Alignment of transcriptomes w/ Pipseeker, STAR
    #Python
    #R

In [ ]:
FIG 1-2, S1: Barcode and transcriptome alignment done in terminal

In [ ]:
#Terminal set up
#Activate virtual environment
source /Users/maddieurbanek/Desktop/Rabies/localanalysis/myenv/bin/activate​

#Load path of samtools (un-hash and change to your path if you want to use this)
#export PATH=$PATH:path/to/samtools/for/server
​
#Load path of bbduk (change to your path)
export PATH=$PATH:/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/
​
#Load py3 environment which contains newest pip version of umi_tools
export PATH=/Users/maddieurbanek/Desktop/Rabies/localanalysis/myenv:$PATH

In [ ]:
#Calling barcode bits in terminal
##Generate fastqs with actual CBCs in read 1
/Users/maddieurbanek/pipseeker/pipseeker barcode --fastq . --chemistry v4 --output . --skip-version-check
cd barcoded_fastqs
gunzip *.gz
cat barcoded_1_R1.fastq barcoded_2_R1.fastq>barcode_temp.fq
cat barcode_temp.fq barcoded_3_R1.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_4_R1.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_5_R1.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_6_R1.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_7_R1.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_8_R1.fastq > barcode_R1.fq

cat barcoded_1_R2.fastq barcoded_2_R2.fastq>barcode_temp.fq
cat barcode_temp.fq barcoded_3_R2.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_4_R2.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_5_R2.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_6_R2.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_7_R2.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_8_R2.fastq > barcode_R2.fq
rm *.fastq
rm *temp*

##Make dataset-specific CBC reference genome
cd ../../barcodes
cp barcode_whitelist.txt barcodes.txt   
awk '1;1' barcodes.txt >cbc.txt
awk 'NR % 2 == 1 {sub(/^/,">")} {print}' cbc.txt >cbc.fa  
bowtie2-build cbc.fa cbc 
rm barcodes.txt  
rm cbc.txt  

##Generate rabies barcode and helper count matrices
bbduk.sh in=barcodes_R2.fastq out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq
rm bit1.sam
rm bit1sorted.sam

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq
rm bit2.sam
rm bit2sorted.sam

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt
rm temp1.fq
rm bit3.sam
rm bit3sorted.sam

bbduk.sh in=barcodes_R1.fastq out=cbc.fq ftr=15 qin=33
bowtie2 -x ../barcodes/cbc cbc.fq -N 1 -S cbc.sam
samtools sort cbc.sam -n -o cbcsorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$10}' cbcsorted.sam > cbc.txt


bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc.txt > temp && mv temp cbc.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc.txt > part3.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,1.5,2.2 part3.txt cbc_Insert_UMI.txt > full.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$6"_"$5,$2"-"$3"-"$4}' full.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

bbduk.sh in=barcodes_R2.fastq out=temp.fq maq=20 int=f
bbduk.sh in=temp.fq literal=AGCCATCTGTTGTTT k=12 skipr1 restrictright=125 hdist=1 out=helper.fq outm1=helper_R1.fq outm2=helper_R2.fq int=t
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=helper_R2.fq out=helper_R2.sam int=f
samtools sort helper_R2.sam -n -o helper_R2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' helper_R2sorted.sam  > helper.txt

awk '!/@/' helper.txt > temp && mv temp helperhit.txt  
join -1 1 -2 1 -o 1.1,1.2,2.2 cbc.txt helperhit.txt > helperindex.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 helperindex.txt cbc_Insert_UMI.txt > helperindexfull.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$4"_"$2,$3}' helperindexfull.txt > helperindex.txt
awk 'BEGIN {OFS="\t"}; {print $1,"helper"}' helperindex.txt > helperUMI.txt
umi_tools count_tab --per-cell --edit-distance-threshold=1 -I helperUMI.txt -S helperindex.tsv -L counts.log
awk '!/cell/' helperindex.tsv > tmpfile && mv tmpfile helperindex.tsv
tail -n +2 helperindex.tsv > helpercounts.tsv
{ printf 'CBC\thelper\tUMI_Count\n'; cat helpercounts.tsv; } > helperindex.tsv

In [ ]:
#Sample barcode diversity
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam

bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../3
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../4
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../5
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../6
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../7
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../e1
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../e2
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../e3
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../e4
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../e5
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

cd ../e6
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

../e7
bbduk.sh in=barcodes_R2.fastq.gz out=temp.fq maq=20 int=f

bbduk.sh in=temp.fq out=temp1.fq ftl=19 ftr=38
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit3r temp1.fq -N 1 -S bit1.sam
samtools sort bit1.sam -n -o bit1sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit1sorted.sam > bit1_hit.txt
rm temp1.fq                                   
rm bit1.sam                                                                                                 
rm bit1sorted.sam                          

bbduk.sh in=temp.fq out=temp1.fq ftl=39 ftr=58
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit2r temp1.fq -N 1 -S bit2.sam
samtools sort bit2.sam -n -o bit2sorted.sam
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit2sorted.sam > bit2_hit.txt
rm temp1.fq                                              
rm bit2.sam                                      
rm bit2sorted.sam                        

bbduk.sh in=temp.fq out=temp1.fq ftl=59 ftr=78
bowtie2 -x /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/bits/bit1r temp1.fq -N 1 -S bit3.sam
samtools sort bit3.sam -n -o bit3sorted.sam                
awk 'BEGIN {OFS="\t"}; {print $1,$3}' bit3sorted.sam > bit3_hit.txt                                
rm temp1.fq                            
rm bit3.sam                                                           
rm bit3sorted.sam
bbduk.sh in=barcodes_R1.fastq out=temp4.fq ftr=28 qin=33
/Users/maddieurbanek/Desktop/Rabies/localanalysis/bbmap/reformat.sh in=temp4.fq out=temp4.sam int=f
samtools sort temp4.sam -n -o temp5.sam
awk 'BEGIN {OFS="\t"}; {print $1,$11}' temp5.sam  > cbc_Insert_UMI.txt
    
awk '!/@/' bit1_hit.txt > temp && mv temp bit1_hit.txt
awk '!/@/' bit2_hit.txt > temp && mv temp bit2_hit.txt
awk '!/@/' bit3_hit.txt > temp && mv temp bit3_hit.txt
awk '!/@/' cbc_Insert_UMI.txt > temp && mv temp cbc_Insert_UMI.txt
    
join -1 1 -2 1 -o 1.1,1.2,2.2 bit1_hit.txt bit2_hit.txt > part1.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,2.2 part1.txt bit3_hit.txt > part2.txt
join -1 1 -2 1 -o 1.1,1.2,1.3,1.4,2.2 part2.txt cbc_Insert_UMI.txt > part3.txt
awk 'BEGIN {OFS="\t"}; {print $1"_"$5"_AAAAAAAAAAAA",$2"-"$3"-"$4}' part3.txt > full1.txt
awk '!/\*/' full1.txt > full.txt
sort -k2  full.txt > flat.txt

umi_tools count_tab --per-cell --edit-distance-threshold=1 -I flat.txt -S counts.tsv -L counts.log
tail -n +2 counts.tsv > completecounts.tsv
awk '!/cell/' completecounts.tsv > tmpfile && mv tmpfile completecounts.tsv
{ printf 'CBC\tbarcode\tUMI_Count\n'; cat completecounts.tsv; } > counts.tsv
mv counts.tsv completecounts.tsv

In [ ]:
#Transcriptome alignment
gunzip *.gz
/Users/maddieurbanek/pipseeker/pipseeker full --fastq /Users/maddieurbanek/Downloads/cvs_h/. --star-index-path /Users/maddieurbanek/pipseeker/pipseeker-gex-reference-GRCh38-2022.04 --chemistry v4 --output /Users/maddieurbanek/Downloads/cvs_h/ --retain-barcoded-fastqs --skip-version-check --skip-preflight-check
cd barcodes
awk '1;1' barcode_whitelist.txt > cbc.txt
awk 'NR % 2 == 1 {sub(/^/,">")} {print}' cbc.txt >cbc.fa
cd ../barcoded_fastqs
cd barcoded_fastqs
gunzip *.gz
cat barcoded_1_R1.fastq barcoded_2_R1.fastq>barcode_temp.fq
cat barcode_temp.fq barcoded_3_R1.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_4_R1.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_5_R1.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_6_R1.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_7_R1.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_8_R1.fastq > barcode_R1.fq

cat barcoded_1_R2.fastq barcoded_2_R2.fastq>barcode_temp.fq
cat barcode_temp.fq barcoded_3_R2.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_4_R2.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_5_R2.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_6_R2.fastq > barcode_temp.fq
cat barcode_temp.fq barcoded_7_R2.fastq > barcode_temp1.fq
cat barcode_temp1.fq barcoded_8_R2.fastq > barcode_R2.fq
rm *.fastq
rm *temp*

/usr/local/bin/STAR --genomeDir /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/human_star --sjdbGTFfile /Users/maddieurbanek/Desktop/Rabies/localanalysis/refgenomes/human_star/hg38.ncbiRefSeq.gtf --runThreadN 12 --soloType CB_UMI_Simple --soloCBstart 1 --soloCBlen 16 --soloUMIstart 17 --outSAMattributes CB CR CY GX GN UB UR UY NH HI nM AS sF --outSAMtype BAM SortedByCoordinate --soloCBmatchWLtype Exact --soloUMIdedup 1MM_CR --soloFeatures Gene Velocyto --soloMultiMappers EM --soloCellReadStats Standard --soloCellFilter EmptyDrops_CR --soloUMIfiltering MultiGeneUMI_CR --outSAMunmapped Within --soloBarcodeReadLength 0 --soloCBwhitelist /Users/maddieurbanek/Downloads/cvs_l/barcodes/barcode_whitelist.txt --soloUMIlen 12 --readFilesIn /Users/maddieurbanek/Downloads/cvs_l/barcoded_fastqs/barcode_R2.fq /Users/maddieurbanek/Downloads/cvs_l/barcoded_fastqs/barcode_R1.fq --outFileNamePrefix trimmed_


In [ ]:
#Load in modules:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy as scipy
from itertools import product
from pycirclize import Circos
import seaborn as sns
from statsmodels.multivariate.manova import MANOVA

In [ ]:
FIG 4: Transcriptome and barcode-coupled analyses

In [ ]:
#Connectivity analysis (w/ conservative barcode calling)
metadata=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/metadata.csv',delimiter=',')
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/thresholded_barcodes.csv',delimiter=',')
foundercells=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/thresholded_helpers.csv',delimiter=',')
barcodes.drop(barcodes[barcodes['UMI_Count'] < 3].index, inplace=True)
barcodes = barcodes[barcodes['CBC'].isin(metadata['cellbarcode'])]
foundercells.drop(foundercells[foundercells['UMI_Count'] < 3].index, inplace=True)
print(len(barcodes))
l23=metadata.loc[metadata['celltype'] == 'L2/3', 'cellbarcode']
l4=metadata.loc[metadata['celltype'] == 'L4', 'cellbarcode']
rg=metadata.loc[metadata['celltype'] == 'RG', 'cellbarcode']
inh=metadata.loc[metadata['celltype'] == 'In', 'cellbarcode']
pro=metadata.loc[metadata['celltype'] == 'Progenitors', 'cellbarcode']
ast=metadata.loc[metadata['celltype'] == 'Astrocytes', 'cellbarcode']
l56sp=metadata.loc[metadata['celltype'] == 'L5/6/SP', 'cellbarcode']
opcs=metadata.loc[metadata['celltype'] == 'OPCs', 'cellbarcode']

barcodes['celltype'] = barcodes.loc[:, 'CBC']
barcodes.loc[barcodes['CBC'].isin(l23), 'celltype'] = 'L2/3'
barcodes.loc[barcodes['CBC'].isin(l4), 'celltype'] = 'L4'
barcodes.loc[barcodes['CBC'].isin(rg), 'celltype'] = 'RG'
barcodes.loc[barcodes['CBC'].isin(inh), 'celltype'] = 'In'
barcodes.loc[barcodes['CBC'].isin(pro), 'celltype'] = 'Progenitors'
barcodes.loc[barcodes['CBC'].isin(ast), 'celltype'] = 'Astrocytes'
barcodes.loc[barcodes['CBC'].isin(l56sp), 'celltype'] = 'L5/6/SP'
barcodes.loc[barcodes['CBC'].isin(opcs), 'celltype'] = 'OPCs'
actualbarcodes = barcodes[barcodes.duplicated(subset=['barcode'], keep=False)]
print('Total number of barcodes with at least one pair:')
print(len(actualbarcodes))

actualbarcodes['helper'] = actualbarcodes.loc[:, 'CBC']
foundercells=foundercells['CBC']
print('Number of founders in dataset:')
print(len(foundercells))
print('Total number of cells with barcodes in dataset:')
print(actualbarcodes['CBC'].nunique())
actualbarcodes.loc[actualbarcodes['CBC'].isin(foundercells), 'helper'] = 'founder'
actualbarcodes.loc[~actualbarcodes['CBC'].isin(foundercells), 'helper'] = 'presynaptic'


cellprops=actualbarcodes.groupby(['celltype','helper'])['CBC'].nunique()
#actualbarcodes.to_csv('/Users/maddieurbanek/Desktop/rabies_project/data/finalized_barcodes.csv')


In [ ]:
sad_1=actualbarcodes[actualbarcodes['CBC'].str.contains('s1')] 
sad_2=actualbarcodes[actualbarcodes['CBC'].str.contains('s2')] 
sad_3=actualbarcodes[actualbarcodes['CBC'].str.contains('s3')] 
sad_4=actualbarcodes[actualbarcodes['CBC'].str.contains('s4')] 
sad_5=actualbarcodes[actualbarcodes['CBC'].str.contains('s5')] 
cvs_1=actualbarcodes[actualbarcodes['CBC'].str.contains('c1')] 
cvs_2=actualbarcodes[actualbarcodes['CBC'].str.contains('c2')] 
nuc_1=actualbarcodes[actualbarcodes['CBC'].str.contains('c1')] 
nuc_2=actualbarcodes[actualbarcodes['CBC'].str.contains('c2')] 

sad_1_props=sad_1.groupby(['celltype','helper'])['CBC'].nunique()
sad_2_props=sad_2.groupby(['celltype','helper'])['CBC'].nunique()
sad_3_props=sad_3.groupby(['celltype','helper'])['CBC'].nunique()
sad_4_props=sad_4.groupby(['celltype','helper'])['CBC'].nunique()
sad_5_props=sad_5.groupby(['celltype','helper'])['CBC'].nunique()
cvs_1_props=cvs_1.groupby(['celltype','helper'])['CBC'].nunique()
cvs_2_props=cvs_2.groupby(['celltype','helper'])['CBC'].nunique()
nuc_1_props=nuc_1.groupby(['celltype','helper'])['CBC'].nunique()
nuc_2_props=nuc_2.groupby(['celltype','helper'])['CBC'].nunique()

print(sad_1_props)
print(sad_2_props)
print(sad_3_props)
print(sad_4_props)
print(sad_5_props)
print(cvs_1_props)
print(cvs_2_props)
print(nuc_1_props)
print(nuc_2_props)

In [ ]:
#Barplot of cell type proportions for each dataset
#Error calculated in 'pre_post_proportions_error.csv'
#strain_metadata.groupby('datasetid')['celltype'].value_counts().to_csv('/Users/maddieurbanek/Desktop/rabies_project/data/pre_post_proportions_error.csv')

x = np.arange(8)
width= 0.20

inf_error = [0.616869773,0.383114866,0.650025879,0.677892149,1.573296212,2.512127079,0.851389428,0.999679136]
founder_error = [1.397144377,0.565818183,1.957174726,0.466496559,0.86925483,5.026498599,1.733980631,6.367147859]

infected=[5.084987054,2.880549452,5.091898166,3.161320824,9.037991071,60.45847935,9.713122485,4.571651599]
founder=[2.160026468,1.720657519,3.745658808,1.071288326,3.843583051,65.33941011,6.151091668,15.96828405]
uninfected=[20.72871657,3.001284597,10.67382927,11.81828798,4.904823076,38.59628635,6.855074156,3.421698003]


plt.bar(x-0.2, founder,width,yerr=founder_error, color='green') 
plt.bar(x, infected,width,yerr=inf_error, color='#d781af')
plt.bar(x+0.2, uninfected,width, color='darkgrey')

plt.xticks(x, ['RGs', 'OPCs', 'Astrocytes','Progenitors','In Neurons','L2/3','L4','L5/6/SP'],rotation=45) 
#plt.xlabel("Cell Types")
plt.ylabel("Percentage of Cells") 
#plt.title('Cell Type Proportions')
#plt.legend(["Rabies+",'Helper+/Rabies+','Uninfected'],loc='upper left',fontsize="10") 
plt.draw()
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/4_transcriptome/starter_cell_type_proportions.svg', bbox_inches='tight', format='svg')
plt.show()

print('Post hoc')
print('RG')
print('inf')
inf=[0.054347826,0.06950431,0.077210461,0.03127506,	0.028503563,	0.066037736,	0.032366071,	0.066037736,	0.032366071]
print(stats.ttest_1samp(inf, popmean=0.207287166))
print(9.98E-09*16)
print()
print('founder')
founder=[0,	0,	0.138211382,	0.006493506,	0.011235955,	0.019230769,	0,	0.019230769,	0]
print(stats.ttest_1samp(founder, popmean=0.207287166))
print(1.54E-06*16)
print()

print('Post hoc')
print('OPCs')
print('inf')
inf=[0.038043478	,0.005926724	,0.04109589,	0.044105854,	0.032066508,	0.029753266	,0.019252232,	0.029753266	,0.019252232]
print(stats.ttest_1samp(founder, popmean=0.030012846))
print(0.58581971*16)
print()
print('founder')
founder=[0	,0	,0.028455285	,0.058441558	,0.011235955	,0.019230769	,0.00913242	,0.019230769	,0.00913242]
print(stats.ttest_1samp(founder, popmean=0.030012846))
print(0.065396927*16)
print()

print('Post hoc')
print('Astrocytes')
print('inf')
inf=[0.027173913,	0.045258621,	0.073474471,	0.052927025,	0.042755344,	0.077648766,0.030691964	,0.077648766	,0.030691964]
print(stats.ttest_1samp(founder, popmean=0.106738293))
print(4.02E-07*16)
print()
print('founder')
founder=[0.0625	,0,	0.191056911,	0.051948052,	0.02247191,	0,	0.00456621,	0,	0.00456621]
print(stats.ttest_1samp(founder, popmean=0.106738293))
print(0.01027025*16)
print()


print('Post hoc')
print('Progenitors')
print('inf')
inf=[0.038043478	,0.027478448,	0.085927771,	0.017642342	,0.013064133,	0.028301887	,0.022879464	,0.028301887,	0.022879464]
print(stats.ttest_1samp(founder, popmean=0.11818288))
print(0.004617419*16)
print()
print('founder')
founder=[0	,0	,0.032520325	,0.038961039,	0.011235955,	0	,0.006849315,	0	,0.006849315]
print(stats.ttest_1samp(founder, popmean=0.11818288))
print(2.13E-08*16)
print()


print('Post hoc')
print('GABAergic')
print('inf')
inf=[0.10326087,	0.041487069,	0.204234122,	0.086607859,	0.097387173,	0.043541364,	0.096679688,	0.043541364,	0.096679688]
print(stats.ttest_1samp(founder, popmean=0.049048231))
print(5.50E-05*16)
print()
print('founder')
founder=[0.0625,	0,	0.040650407,	0.038961039,	0,	0.028846154,	0.073059361,	0.028846154,	0.073059361]
print(stats.ttest_1samp(founder, popmean=0.049048231))
print(0.282954751*16)
print()


print('Post hoc')
print('L2/3')
print('inf')
inf=[0.66576087	,0.643857759	,0.434620174	,0.610264635	,0.660332542	,0.547895501,	0.66531808	,0.547895501,	0.66531808]
print(stats.ttest_1samp(founder, popmean=0.385962863))
print(2.69E-10*16)
print()
print('founder')
founder=[0.84375,	0.454545455,	0.430894309,	0.746753247,	0.842696629,	0.548076923,	0.732876712,	0.548076923,	0.732876712]
print(stats.ttest_1samp(founder, popmean=0.385962863))
print(0.00103182*16)
print()


print('Post hoc')
print('L4')
print('inf')
inf=[0.073369565,	0.106681034,	0.067247821,	0.076182839,	0.062945368,	0.129898403,	0.113978795,	0.129898403,	0.113978795]
print(stats.ttest_1samp(founder, popmean=0.068550742))
print(4.24E-06*16)
print()
print('founder')
founder=[0,	0,	0.138211382,	0.032467532,	0.078651685,	0.028846154,	0.123287671,	0.028846154,	0.123287671]
print(stats.ttest_1samp(founder, popmean=0.068550742))
print(0.711857659*16)
print()


print('Post hoc')
print('L5/6/SP')
print('inf')
inf=[0,	0.059806034	,0.01618929	,0.080994387	,0.062945368,	0.076923077,	0.018833705,	0.076923077,	0.018833705]
print(stats.ttest_1samp(founder, popmean=0.03421698))
print(0.176092512*16)
print()
print('founder')
founder=[0.03125,	0.545454545,	0,	0.025974026,	0.02247191,	0.355769231,	0.050228311,	0.355769231,	0.050228311]
print(stats.ttest_1samp(founder, popmean=0.03421698))
print(0.100265239*16)
print()


In [ ]:
FIG 5: CVS-N2c vs. SAD B19

In [ ]:
strain_metadata=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/strains_metadata.csv',delimiter=',')
print(strain_metadata.groupby('virus')['celltype'].value_counts())

In [ ]:
#Transcriptome features ANOVA and Wilcoxon Rank Sum
cvs=strain_metadata.loc[strain_metadata['virus'] == 'cvs', ['nCount_RNA','nFeature_RNA','percent.mt']]
sad=strain_metadata.loc[strain_metadata['virus'] == 'sadb19', ['nCount_RNA','nFeature_RNA','percent.mt']]
uninfected=strain_metadata.loc[strain_metadata['virus'] == 'uninfected', ['nCount_RNA','nFeature_RNA','percent.mt']]

In [ ]:
#One sample t-test
#Proportions can be found in strain_proportions_error.csv
print('Overall Paired T-test:CVS')


print()
print('Post hoc')
print('RG')
cvs=[0.064014916,0.03260743]
print(stats.ttest_1samp(cvs, popmean=0.226564598))
print(0.055940407614042606*16)
print()

print('OPCs')
cvs=[0.028589186,0.019098637]
print(stats.ttest_1samp(cvs, popmean=0.030083266))
print(0.4139379611523105*16)
print()

print('Astrocytes')
cvs=[0.072715973,0.030394783]
print(stats.ttest_1samp(cvs, popmean=0.115901155))
print(0.20226489946514745*16)
print()

print('Progenitors')
cvs=[0.027346178,0.023873297]
print(stats.ttest_1samp(cvs, popmean=0.128659683))
print(0.010726330241811608*16)
print()

print('In')
cvs=[0.040397763,0.095376732]
print(stats.ttest_1samp(cvs, popmean=0.042035992))
print(0.5195460588099319*16)
print()

print('L2/3')
cvs=[0.546302051,0.667986491]
print(stats.ttest_1samp(cvs, popmean=0.355627182))
print(0.15109636302320864*16)
print()

print('L4')
cvs=[0.121193288,0.111563992]
print(stats.ttest_1samp(cvs, popmean=0.065001343))
print(0.05948492517796917*16)
print()

print('L5/6/SP')
cvs=[0.099440646,0.019098637]
print(stats.ttest_1samp(cvs, popmean=0.036126779))
print(0.6672598228794371*16)
print()

#One sample t-test
#Proportions can be found in strain_proportions_error.csv
print('Overall Paired T-test:SADB19')

print()
print('Post hoc')
print('RG')
cvs=[0.05,0.069094804,0.095070423,0.028348689,0.025341131]
print(stats.ttest_1samp(cvs, popmean=0.226564598))
print(0.00018746514577334662*16)
print()

print('OPCs')
cvs=[0.035,0.005891805,0.037852113,0.046066619,0.029239766]
print(stats.ttest_1samp(cvs, popmean=0.030083266))
print(0.9199641518326075*16)
print()

print('Astrocytes')
cvs=[0.03,0.044991966,0.094190141,0.052445074,0.038986355]
print(stats.ttest_1samp(cvs, popmean=0.115901155))
print(0.004609400962170117*16)
print()

print('Progenitors')
cvs=[0.035,0.027316551,0.070422535,0.021261517,0.012670565]
print(stats.ttest_1samp(cvs, popmean=0.128659683))
print(0.000667965607907446*16)
print()

print('In')
cvs=[0.1,0.041242635,0.167253521,0.080793763,0.079922027]
print(stats.ttest_1samp(cvs, popmean=0.042035992))
print(0.06644877394436453*16)
print()

print('L2/3')
cvs=[0.68,0.642742367,0.44278169,0.625797307,0.692007797]
print(stats.ttest_1samp(cvs, popmean=0.355627182))
print(0.004429244040512132*16)
print()

print('L4')
cvs=[0.0675,0.106052491,0.080985915,0.070871722,0.065302144]
print(stats.ttest_1samp(cvs, popmean=0.065001343))
print(0.15364983934402618*16)
print()

print('L5/6/SP')
cvs=[0.0025,0.062667381,0.011443662,0.074415308,0.056530214]
print(stats.ttest_1samp(cvs, popmean=0.036126779))
print(0.7285018930227892*16)
print()

In [ ]:
#Barplot of cell type proportions for each dataset
#Error calculated in 'strain_proportions_error.csv'
#strain_metadata.groupby('datasetid')['celltype'].value_counts().to_csv('/Users/maddieurbanek/Desktop/rabies_project/data/strain_proportions_error.csv')

x = np.arange(8)
width= 0.20

sad_error = [1.305571354,0.679446084,1.114136055,0.996811688,2.068560815,4.510359902,0.747681196,1.445988536]
cvs_error = [1.570374313,0.474527418,2.116059492,0.173644045,2.748948484,6.084222012,0.481464784,4.017100444]


cvs=[(0.048311173)*100,(0.023843912)*100,(0.051555378)*100,(0.025609737)*100,(0.067887247)*100,(0.607144271)*100,(0.11637864)*100,(0.059269642)*100]
sad=[(0.053571009)*100,(0.030810061)*100,(0.052122707)*100,(0.033334234)*100,(0.093842389)*100,(0.616665832)*100,(0.078142455)*100,(0.041511313)*100]
uninfected=[(1687/7446)*100,(224/7446)*100,(863/7446)*100,(958/7446)*100,(313/7446)*100,(2648/7446)*100,(484/7446)*100,(269/7446)*100]

plt.bar(x-0.2, cvs,width, yerr=cvs_error,color='#5b1a18')
plt.bar(x, sad,width, yerr=sad_error,color='#fd6467') 
plt.bar(x+0.2, uninfected,width, color='#f1bb7b')

plt.xticks(x, ['RGs', 'OPCs', 'Astrocytes','IPCs','In Neurons','L2/3','L4','L5/6/SP'],rotation=45) 
#plt.xlabel("Cell Types")
plt.ylabel("Percentage of Cells") 
#plt.title('Cell Type Proportions')
#plt.legend(["Rabies+, Cell",'Helper+/Rabies+, Cell',"Rabies+, Nuc",'Helper+/Rabies+, Nuc'],loc='upper left',fontsize="10") 
plt.draw()
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/5_sad_cvs/cell_type_proportions.svg', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
#Ephys bar plots
#Barplot of cell type proportions for each dataset

sad=[55,11]
cvs=[((21/73)*66),((52/73)*66)]
import scipy.stats as stats 
# creating data 
data = [sad, cvs] 
# performing fishers exact test on the data 
odd_ratio, p_value = stats.fisher_exact(data) 
print('odds ratio is : ' + str(odd_ratio)) 
print('p_value is : ' + str(p_value)) 
#post hoc fishers

plt.figure().set_figwidth(3)
x = np.arange(2)
width= 0.2

sad=[83.33333333,16.66666667]
cvs=[28.76712329,71.23287671]


#Clopper pearson error bars
print('Success 95% CI')
print('SADB19')
from scipy.stats import binomtest
result = binomtest(k=11, n=66, p=0.05)
print(result.proportion_ci())
print('CVS')
result = binomtest(k=52, n=73, p=0.05)
print(result.proportion_ci())

print('Failure 95% CI')
print('SADB19')
from scipy.stats import binomtest
result = binomtest(k=55, n=66, p=0.05)
print(result.proportion_ci())
print('CVS')
result = binomtest(k=21, n=73, p=0.05)
print(result.proportion_ci())


sad_lower_error = [(((55/66)*100)-(0.7213265468376223*100)),(((11/66)*100)-(0.0862479340483983*100))]
sad_upper_error = [((0.91375206595160213*100)-((55/66)*100)),((0.27867345316237774*100)-((11/66)*100))]
sad_asymmetric_error = [sad_lower_error, sad_upper_error]


cvs_lower_error = [(((21/73)*100)-(0.18769257480984877*100)),(((51/73)*100)-(0.5944688763766685*100))]
cvs_upper_error = [((0.40553112362333144*100)-((21/73)*100)),((0.8123074251901511*100)-((51/73)*100))]
cvs_asymmetric_error = [cvs_lower_error, cvs_upper_error]


plt.bar(x-0.1, sad,width, yerr=sad_asymmetric_error,color='#fd6467')
plt.bar(x+0.1, cvs,width, yerr=cvs_asymmetric_error,color='#5b1a18')


plt.xticks(x, ['No AP', 'AP']) 
#plt.xlabel("Cell Types")
plt.ylabel("Percentage of Cells") 
#plt.title('Cell Type Proportions')
#plt.legend(["Rabies+, Cell",'Helper+/Rabies+, Cell',"Rabies+, Nuc",'Helper+/Rabies+, Nuc'],loc='upper left',fontsize="10") 
plt.draw()
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/5_sad_cvs/ephys_proportions.svg', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
#Ephys variance plots
ephys=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/ephys_properties.csv',delimiter=',')
cvs_ephys=ephys.loc[ephys['strain'] == 'cvs']
sad_ephys=ephys.loc[ephys['strain'] == 'sad']

#RMP
print(cvs_ephys['Resting Membrane potential (mV)'].min())
print(sad_ephys['Resting Membrane potential (mV)'].min())
print(cvs_ephys['Resting Membrane potential (mV)'].max())
print(sad_ephys['Resting Membrane potential (mV)'].max())

print(cvs_ephys['Resting Membrane potential (mV)'].mean())
print(cvs_ephys['Resting Membrane potential (mV)'].median())

print(sad_ephys['Resting Membrane potential (mV)'].mean())
print(sad_ephys['Resting Membrane potential (mV)'].median())

print('Wilcoxon Ranked Sum to compare distributions for RMP:')
import numpy as np
from scipy.stats import ranksums
cvs_dist = cvs_ephys['Resting Membrane potential (mV)']
sad_dist = sad_ephys['Resting Membrane potential (mV)']
print(ranksums(cvs_dist, sad_dist))

print('Welchs T-test to compare means for RMP:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(cvs_dist, sad_dist,equal_var=False))

sns.histplot(data=cvs_ephys, x='Resting Membrane potential (mV)',binwidth=5, binrange=(-80.5, -2.0),kde=True,color='#5b1a18')
sns.histplot(data=sad_ephys, x='Resting Membrane potential (mV)', binwidth=5,binrange=(-80.5, -2.0),kde=True,color='#fd6467')

plt.xlim([-90, 0])
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/5_sad_cvs/rmp_variance.svg', bbox_inches='tight', format='svg')
plt.show()

#CC
print(cvs_ephys['Cell Capacitance (pF)'].min())
print(sad_ephys['Cell Capacitance (pF)'].min())
print(cvs_ephys['Cell Capacitance (pF)'].max())
print(sad_ephys['Cell Capacitance (pF)'].max())

print(cvs_ephys['Cell Capacitance (pF)'].mean())
print(cvs_ephys['Cell Capacitance (pF)'].median())

print(sad_ephys['Cell Capacitance (pF)'].mean())
print(sad_ephys['Cell Capacitance (pF)'].median())

print('Wilcoxon Ranked Sum to compare distributions for CC:')
import numpy as np
from scipy.stats import ranksums
cvs_dist = cvs_ephys['Cell Capacitance (pF)']
sad_dist = sad_ephys['Cell Capacitance (pF)']
print(ranksums(cvs_dist, sad_dist))

print('Welchs T-test to compare means for CC:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(cvs_dist, sad_dist,equal_var=False))

sns.histplot(data=cvs_ephys, x='Cell Capacitance (pF)', binwidth=5, binrange=(6.289, 91.82),kde=True,color='#5b1a18')
sns.histplot(data=sad_ephys, x='Cell Capacitance (pF)', binwidth=5, binrange=(6.289, 91.82),kde=True,color='#fd6467')

plt.xlim([0, 100])
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/5_sad_cvs/cc_variance.svg', bbox_inches='tight', format='svg')
plt.show()

#IR
print(cvs_ephys['Input Resistance (MΩ)'].min())
print(sad_ephys['Input Resistance (MΩ)'].min())
print(cvs_ephys['Input Resistance (MΩ)'].max())
print(sad_ephys['Input Resistance (MΩ)'].max())

print(cvs_ephys['Input Resistance (MΩ)'].mean())
print(cvs_ephys['Input Resistance (MΩ)'].median())

print(sad_ephys['Input Resistance (MΩ)'].mean())
print(sad_ephys['Input Resistance (MΩ)'].median())

print('Wilcoxon Ranked Sum to compare distributions for IR:')
import numpy as np
from scipy.stats import ranksums
cvs_dist = cvs_ephys['Input Resistance (MΩ)']
sad_dist = sad_ephys['Input Resistance (MΩ)']
print(ranksums(cvs_dist, sad_dist))

print('Welchs T-test to compare means for IR:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(cvs_dist, sad_dist,equal_var=False))

sns.histplot(data=cvs_ephys, x='Input Resistance (MΩ)', binwidth=500, binrange=(22.7, 4100.0),kde=True,color='#5b1a18')
sns.histplot(data=sad_ephys, x='Input Resistance (MΩ)', binwidth=500, binrange=(22.7, 4100.0),kde=True,color='#fd6467')


plt.xlim([0, 4500])
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/5_sad_cvs/ir_variance.svg', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
FIG 6: Cellular barcodes vs. nuclear barcodes comparison

In [ ]:
nuc_metadata=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/metadata.csv',delimiter=',')
cvs_datasets=['c1','c2','n1','n2']
nuc_metadata = nuc_metadata.loc[nuc_metadata['datasetid'].isin(cvs_datasets)]
print(nuc_metadata.groupby('modality')['celltype'].value_counts())


In [ ]:
n1_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/barcode_nuc/barcoded_fastqs/completecounts.tsv',delimiter='\t')
n2_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/barcode_nuc/barcoded_fastqs/completecounts.tsv',delimiter='\t')

c1_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/barcode_cell/barcoded_fastqs/completecounts.tsv',delimiter='\t')
c2_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/barcode_cell/barcoded_fastqs/completecounts.tsv',delimiter='\t')


n1_barcodes['CBC'] = n1_barcodes['CBC'].str.replace("'", "")
n1_barcodes['CBC'] = n1_barcodes['CBC'].str.replace('b', 'n1_')
n2_barcodes['CBC'] = n2_barcodes['CBC'].str.replace("'", "")
n2_barcodes['CBC'] = n2_barcodes['CBC'].str.replace('b', 'n2_')


c1_barcodes['CBC'] = c1_barcodes['CBC'].str.replace("'", "")
c1_barcodes['CBC'] = c1_barcodes['CBC'].str.replace('b', 'c1_')
c2_barcodes['CBC'] = c2_barcodes['CBC'].str.replace("'", "")
c2_barcodes['CBC'] = c2_barcodes['CBC'].str.replace('b', 'c2_')

nuc_barcodes = pd.concat([n1_barcodes, n2_barcodes], axis=0, ignore_index=True)
cell_barcodes = pd.concat([c1_barcodes, c2_barcodes], axis=0, ignore_index=True)
#cell_barcodes.drop(cell_barcodes[cell_barcodes['UMI_Count'] < 3].index, inplace=True)
#nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 3].index, inplace=True)
qc_nuc=nuc_metadata['cellbarcode']

In [ ]:
summed_nuc=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_cell=cell_barcodes.groupby('CBC')['UMI_Count'].sum()

summed_nuc = pd.DataFrame(summed_nuc)
summed_cell = pd.DataFrame(summed_cell)

summed_nuc = summed_nuc.reset_index()
summed_cell = summed_cell.reset_index()


real_nuc = summed_nuc.loc[summed_nuc['CBC'].isin(qc_nuc)]
real_cell = summed_cell.loc[summed_cell['CBC'].isin(qc_nuc)]

In [ ]:
print('Set bins for nuc:')
print((np.log10(nuc_barcodes['UMI_Count'])).min())
print((np.log10(nuc_barcodes['UMI_Count'])).max())
print('Set bins for cell:')
print((np.log10(cell_barcodes['UMI_Count'])).min())
print((np.log10(cell_barcodes['UMI_Count'])).max())
print()

#import numpy as np
#from scipy.stats import ranksums

nuc_dist = nuc_barcodes['UMI_Count']
cell_dist = cell_barcodes['UMI_Count']

nuc_dist.to_csv('/Users/maddieurbanek/Desktop/nuc_dist.csv')
cell_dist.to_csv('/Users/maddieurbanek/Desktop/cell_dist.csv')

print('Nuc mean:')
print(nuc_dist.mean())
print('Nuc median:')
print(nuc_dist.median())
print('Nuc SEM:')
print(nuc_dist.sem())
print('Cell mean:')
print(cell_dist.mean())
print('Cell median:')
print(cell_dist.median())
print('Cell SEM:')
print(cell_dist.sem())

print('Wilcoxon Ranked Sum Results:')
print(ranksums(nuc_dist, cell_dist))
print(len(cell_dist))
print(len(nuc_dist))

In [ ]:
plt.figure().set_figheight(5)
plt.hist(np.log10(cell_barcodes['UMI_Count']),bins=np.arange(0, 3.718916686014861 + .05, .05),color='#7294d4',alpha=0.7)
plt.hist(np.log10(nuc_barcodes['UMI_Count']),bins=np.arange(0, 2.9355072658247128 + .05, .05),color='#e6a0c4',alpha=0.7)

#plt.xlabel('UMIs per Barcode:CBC Combo')
#plt.ylabel('Frequency')
plt.xlim([-.1, 4.5])
plt.yscale('log')
#plt.title('UMIs per Barcode:CBC Combo')
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/6_cell_nuc/combo_dist.svg', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
print('Set bins for nuc:')
print((np.log10(real_nuc['UMI_Count'])).min())
print((np.log10(real_nuc['UMI_Count'])).max())
print('Set bins for cell:')
print((np.log10(real_cell['UMI_Count'])).min())
print((np.log10(real_cell['UMI_Count'])).max())


print()
print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
#import scipy
#from scipy.stats import ranksums

nuc_dist = real_nuc['UMI_Count']
cell_dist = real_cell['UMI_Count']

nuc_dist.to_csv('/Users/maddieurbanek/Desktop/total_nuc_dist.csv')
cell_dist.to_csv('/Users/maddieurbanek/Desktop/total_cell_dist.csv')

print('Nuc mean:')
print((nuc_dist.mean()))
print('Nuc median:')
print((nuc_dist.median()))
print('Nuc SEM:')
print(nuc_dist.sem())
print('Cell mean:')
print(cell_dist.mean())
print('Cell median:')
print((cell_dist.median()))
print('Cell SEM:')
print(cell_dist.sem())

from scipy.stats import ranksums
print('Wilcoxon Ranked Sum Results:')
print(ranksums(nuc_dist, cell_dist))
print(len(nuc_dist))
print(len(cell_dist))

#print('IQR for nuclei')
#print(iqr(nuc_dist))

#print('IQR for cells')
#print(iqr(cell_dist))

In [ ]:
plt.figure().set_figheight(5)
#Set bins to the max values printed above
#Pre-thresholded distribution of barcode UMI counts
plt.hist((np.log10(real_cell['UMI_Count'])),bins=np.arange(0, 3.7712934426290596 + .01, .05),alpha=0.7,color='#7294d4',label='single cell')
plt.hist((np.log10(real_nuc['UMI_Count'])),bins=np.arange(0, 3.109240968588203 + .01, .05),color='#e6a0c4',alpha=0.7, label= 'single nuc')
plt.xlim([-.1, 5])
plt.legend(loc='upper right')
plt.xlabel('Log10(Total Barcode Transcripts per Cell/Nuclei)')
plt.ylabel('Frequency')
plt.yscale('log')
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/6_cell_nuc/barcode_capture.svg', bbox_inches='tight', format='svg')
plt.show()


In [ ]:
#Barcode Inflation QC w/ thresholded dataset
#Barcode UMIs per cell vs. Unique barcodes per cell
real_nuc = nuc_barcodes.loc[nuc_barcodes['CBC'].isin(qc_nuc)]
real_cell = cell_barcodes.loc[cell_barcodes['CBC'].isin(qc_nuc)]

nuc_total_umis=(real_nuc.groupby(['CBC'], as_index=False)['UMI_Count'].sum())
nuc_uniq_bcs=(real_nuc.groupby(['CBC'], as_index=False)['barcode'].nunique())

nuc_mergeddf = pd.merge(nuc_total_umis, nuc_uniq_bcs, on ='CBC')

cell_total_umis=(real_cell.groupby(['CBC'], as_index=False)['UMI_Count'].sum())
cell_uniq_bcs=(real_cell.groupby(['CBC'], as_index=False)['barcode'].nunique())

cell_mergeddf = pd.merge(cell_total_umis, cell_uniq_bcs, on ='CBC')

# plot 1 with axes level-plot
nuc_mergeddf['logged_umi']=np.log10(nuc_mergeddf['UMI_Count'])
nuc_mergeddf['logged_bar']=np.log10(nuc_mergeddf['barcode'])
#ax = sns.regplot(data=nuc_mergeddf, x='logged_umi', y='logged_bar', color = '#e6a0c4')

cell_mergeddf['logged_umi']=np.log10(cell_mergeddf['UMI_Count'])
cell_mergeddf['logged_bar']=np.log10(cell_mergeddf['barcode'])
sns.regplot(data=cell_mergeddf, x='logged_umi', y='logged_bar', color = '#7294d4',scatter_kws={'alpha':0.1})
ax=sns.regplot(data=nuc_mergeddf, x='logged_umi', y='logged_bar', color = '#e6a0c4')


#ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/6_cell_nuc/logged_inflation.svg")

#corr_matrix = np.corrcoef(np.log10(nuc_mergeddf['UMI_Count']), np.log10(nuc_mergeddf['barcode']))
print(scipy.stats.spearmanr(np.log10(nuc_mergeddf['UMI_Count']), np.log10(nuc_mergeddf['barcode']),nan_policy="omit"))
#print(0.9523766792293984**2)
corr = corr_matrix[0,1]
R_sq = corr**2
print('Nuc r_squared:')
print(R_sq)

#corr_matrix = np.corrcoef(np.log10(cell_mergeddf['UMI_Count']), np.log10(cell_mergeddf['barcode']))
print(scipy.stats.spearmanr(np.log10(cell_mergeddf['UMI_Count']), np.log10(cell_mergeddf['barcode']),nan_policy="omit"))
print('Cell r_squared:')
#print(0.8138803974603327**2)
corr = corr_matrix[0,1]
R_sq = corr**2
print('Cell r_squared:')
print(R_sq)

In [ ]:
BARCODE DIVERSITY

In [ ]:
#plasmid comparisons
c_2=pd.read_table('/Users/maddieurbanek/Downloads/cvs_completecounts.tsv',delimiter='\t')
c_1=pd.read_table('/Users/maddieurbanek/Downloads/plasmid_sadb19_completecounts.tsv',delimiter='\t')

print(c_1['UMI_Count'].sum())
print(c_2['UMI_Count'].sum())

print(len(c_1))
print(len(c_2))

c_1['%UMI']=(c_1['UMI_Count']/55230151)*100
c_2['%UMI']=(c_2['UMI_Count']/87549500)*100

c_1_s=c_1.sort_values(by='%UMI',ascending=False)
c_1=c_1_s.reset_index()
c_1['rank']=c_1.index
c_2_s=c_2.sort_values(by='%UMI',ascending=False)
c_2=c_2_s.reset_index()
c_2['rank']=c_2.index


df = pd.DataFrame(index=range(len(c_2)-len(c_1)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
c_1 = pd.concat([c_1, df], ignore_index = True)


plt.plot(np.log10(c_1['rank']),c_1['%UMI'],label = "SADB19",color = '#882255',linewidth=1.5)

plt.plot(np.log10(c_2['rank']),c_2['%UMI'], label='CVS-N2c', color = 'indigo',linewidth=1.5,alpha=0.7)

plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Cell Line")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
plt.axis([0, 8, -0.1, 0.9])
plt.legend(loc='upper right')
plt.draw()
#plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/plasmids.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
#cell line comparison (lipofectamine3000, hekgt vs. hek293t)
h_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cell_line/hekgt/2/completecounts.tsv',delimiter='\t')
h_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cell_line/hekgt/3/completecounts.tsv',delimiter='\t')
h_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cell_line/hekgt/4/completecounts.tsv',delimiter='\t')
h_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cell_line/hekgt/5/completecounts.tsv',delimiter='\t')

l3_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/2/completecounts.tsv',delimiter='\t')
l3_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/3/completecounts.tsv',delimiter='\t')
l3_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/4/completecounts.tsv',delimiter='\t')
l3_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/5/completecounts.tsv',delimiter='\t')

print(h_2['UMI_Count'].sum())
print(h_3['UMI_Count'].sum())
print(h_4['UMI_Count'].sum())
print(h_5['UMI_Count'].sum())

print(l3_2['UMI_Count'].sum())
print(l3_3['UMI_Count'].sum())
print(l3_4['UMI_Count'].sum())
print(l3_5['UMI_Count'].sum())

print(len(h_2))
print(len(h_3))
print(len(h_4))
print(len(h_5))
print(len(l3_2))
print(len(l3_3))
print(len(l3_4))
print(len(l3_5))


h_2['%UMI']=(h_2['UMI_Count']/22531141)*100
h_3['%UMI']=(h_3['UMI_Count']/23469504)*100
h_4['%UMI']=(h_4['UMI_Count']/20661031)*100
h_5['%UMI']=(h_5['UMI_Count']/22821298)*100
l3_2['%UMI']=(l3_2['UMI_Count']/25457517)*100
l3_3['%UMI']=(l3_3['UMI_Count']/15639047)*100
l3_4['%UMI']=(l3_4['UMI_Count']/20028038)*100
l3_5['%UMI']=(l3_5['UMI_Count']/18347023)*100

print('Wilcoxon rank sum')
print(ranksums(h_2, h_3))



h_2_s=h_2.sort_values(by='%UMI',ascending=False)
h_2=h_2_s.reset_index()
h_2['rank']=h_2.index
h_3_s=h_3.sort_values(by='%UMI',ascending=False)
h_3=h_3_s.reset_index()
h_3['rank']=h_3.index
h_4_s=h_4.sort_values(by='%UMI',ascending=False)
h_4=h_4_s.reset_index()
h_4['rank']=h_4.index
h_5_s=h_5.sort_values(by='%UMI',ascending=False)
h_5=h_5_s.reset_index()
h_5['rank']=h_5.index

l3_2_s=l3_2.sort_values(by='%UMI',ascending=False)
l3_2=l3_2_s.reset_index()
l3_2['rank']=l3_2.index
l3_3_s=l3_3.sort_values(by='%UMI',ascending=False)
l3_3=l3_3_s.reset_index()
l3_3['rank']=l3_3.index
l3_4_s=l3_4.sort_values(by='%UMI',ascending=False)
l3_4=l3_4_s.reset_index()
l3_4['rank']=l3_4.index
l3_5_s=l3_5.sort_values(by='%UMI',ascending=False)
l3_5=l3_5_s.reset_index()
l3_5['rank']=l3_5.index

df = pd.DataFrame(index=range(len(l3_5)-len(h_2)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
h_2 = pd.concat([h_2, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l3_5)-len(h_3)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
h_3 = pd.concat([h_3, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l3_5)-len(h_4)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
h_4 = pd.concat([h_4, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l3_5)-len(h_5)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
h_5 = pd.concat([h_5, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l3_5)-len(l3_2)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_2 = pd.concat([l3_2, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l3_5)-len(l3_3)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_3 = pd.concat([l3_3, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l3_5)-len(l3_4)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_4 = pd.concat([l3_4, df], ignore_index = True)


plt.plot(np.log10(h_2['rank']),h_2['%UMI'],label = "HEKGT",color = '#882255',linewidth=1.5)
plt.plot(np.log10(l3_2['rank']),l3_2['%UMI'],label = "HEK293T",color = '#0077BB',linewidth=1.5)
plt.plot(np.log10(h_3['rank']),h_3['%UMI'],color = '#882255',linewidth=1.5,alpha=0.7)
plt.plot(np.log10(l3_3['rank']),l3_3['%UMI'],color = '#0077BB',linewidth=1.5,alpha=0.7)
plt.plot(np.log10(h_4['rank']),h_4['%UMI'],color = '#882255',linewidth=1.5,alpha=0.4)
plt.plot(np.log10(l3_4['rank']),l3_4['%UMI'],color = '#0077BB',linewidth=1.5,alpha=0.4)
plt.plot(np.log10(h_5['rank']),h_5['%UMI'],color = '#882255',linewidth=1.5,alpha=0.2)
plt.plot(np.log10(l3_5['rank']),l3_5['%UMI'],color = '#0077BB',linewidth=1.5,alpha=0.2)


plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Cell Line")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
#plt.axis([0, 7, 0.95, 1000000])
plt.legend(loc='upper right')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/cell_line.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
#old transfection comparison
#cell line comparison (lipofectamine3000, hekgt vs. hek293t)
hek_enva=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/old_transfection/liporealcounts.csv',delimiter=',')
l3_enva=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/old_transfection/xtrerealcounts.csv',delimiter=',')

hek_enva=hek_enva[~hek_enva.barcode.str.contains("Empty")]
l3_enva=l3_enva[~l3_enva.barcode.str.contains("Empty")]

print(hek_enva['UMI_Count'].sum())
print(l3_enva['UMI_Count'].sum())

print(len(hek_enva))
print(len(l3_enva))

hek_enva['%UMI']=(hek_enva['UMI_Count']/53147364)*100
l3_enva['%UMI']=(l3_enva['UMI_Count']/244120961)*100

hek_enva_s=hek_enva.sort_values(by='%UMI',ascending=False)
hek_enva=hek_enva_s.reset_index()
hek_enva['rank']=hek_enva.index

l3_enva_s=l3_enva.sort_values(by='%UMI',ascending=False)
l3_enva=l3_enva_s.reset_index()
l3_enva['rank']=l3_enva.index


df = pd.DataFrame(index=range(len(hek_enva)-len(l3_enva)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_enva = pd.concat([l3_enva, df], ignore_index = True)


plt.plot(np.log10(hek_enva['rank']),hek_enva['%UMI'],label = "Lipofectamine3000",color = '#882255',linewidth=1.5)
plt.plot(np.log10(l3_enva['rank']),l3_enva['%UMI'],label = "XtremeGeneHP",color = '#0077BB',linewidth=1.5)


plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Lipofectamine vs. XtremeGeneHP")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
#plt.axis([0, 7, 0.95, 1000000])
plt.legend(loc='upper right')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/old_transfection_reagents.pdf",bbox_inches='tight', format='pdf')
plt.show()


In [ ]:
#pooled lipofectamine comparison (lipofectamine3000_g, 293 vs. lipofectamine2000_g, 293)
l2_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/2/completecounts.tsv',delimiter='\t')
l2_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/3/completecounts.tsv',delimiter='\t')
l2_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/4/completecounts.tsv',delimiter='\t')
l2_6=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/6/completecounts.tsv',delimiter='\t')

l3_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/2/completecounts.tsv',delimiter='\t')
l3_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/3/completecounts.tsv',delimiter='\t')
l3_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/4/completecounts.tsv',delimiter='\t')
l3_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/5/completecounts.tsv',delimiter='\t')

l2_data=[l2_2,l2_3,l2_4,l2_6]
l2_pooled = pd.concat(l2_data, axis=0, ignore_index=True)
l2_barcodes=l2_pooled.groupby('barcode')['UMI_Count'].sum()
l2_barcodes = pd.DataFrame(l2_barcodes)
l2_barcodes = l2_barcodes.reset_index()

l3_data=[l3_2,l3_3,l3_4,l3_5]
l3_pooled = pd.concat(l3_data, axis=0, ignore_index=True)
l3_barcodes=l3_pooled.groupby('barcode')['UMI_Count'].sum()
l3_barcodes = pd.DataFrame(l3_barcodes)
l3_barcodes = l3_barcodes.reset_index()

print(l2_barcodes['UMI_Count'].sum())
print(l3_barcodes['UMI_Count'].sum())

print(len(l2_barcodes))
print(len(l3_barcodes))


l2_barcodes['%UMI']=(l2_barcodes['UMI_Count']/63093988)*100
l3_barcodes['%UMI']=(l3_barcodes['UMI_Count']/79471625)*100

l2_barcodes_s=l2_barcodes.sort_values(by='%UMI',ascending=False)
l2_barcodes=l2_barcodes_s.reset_index()
l2_barcodes['rank']=l2_barcodes.index

l3_barcodes_s=l3_barcodes.sort_values(by='%UMI',ascending=False)
l3_barcodes=l3_barcodes_s.reset_index()
l3_barcodes['rank']=l3_barcodes.index

df = pd.DataFrame(index=range(len(l2_barcodes)-len(l3_barcodes)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_barcodes = pd.concat([l3_barcodes, df], ignore_index = True)

plt.plot(np.log10(l2_barcodes['rank']),l2_barcodes['%UMI'],label = "Lipofectamine2000",color = '#0077BB',linewidth=1.5)
plt.plot(np.log10(l3_barcodes['rank']),l3_barcodes['%UMI'],label = "Lipofectamine3000",color = '#882255',linewidth=1.5)

plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Pooled Lipofectamine Version")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
#plt.axis([0, 7, 0.95, 1000000])
plt.legend(loc='upper right')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/pooled_lipofectamine_comparison.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
#replicate lipofectamine comparison (lipofectamine3000_g, 293 vs. lipofectamine2000_g, 293)
l2_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/2/completecounts.tsv',delimiter='\t')
l2_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/3/completecounts.tsv',delimiter='\t')
l2_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/4/completecounts.tsv',delimiter='\t')
l2_6=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine2000/6/completecounts.tsv',delimiter='\t')

l3_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/2/completecounts.tsv',delimiter='\t')
l3_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/3/completecounts.tsv',delimiter='\t')
l3_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/4/completecounts.tsv',delimiter='\t')
l3_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/new_transfection/lipofectamine3000/5/completecounts.tsv',delimiter='\t')


print(l2_2['UMI_Count'].sum())
print(l2_3['UMI_Count'].sum())
print(l2_4['UMI_Count'].sum())
print(l2_6['UMI_Count'].sum())

print(l3_2['UMI_Count'].sum())
print(l3_3['UMI_Count'].sum())
print(l3_4['UMI_Count'].sum())
print(l3_5['UMI_Count'].sum())

print(len(l2_2))
print(len(l2_3))
print(len(l2_4))
print(len(l2_6))

print(len(l3_2))
print(len(l3_3))
print(len(l3_4))
print(len(l3_5))

l2_2['%UMI']=(l2_2['UMI_Count']/16098390)*100
l2_3['%UMI']=(l2_3['UMI_Count']/14075623)*100
l2_4['%UMI']=(l2_4['UMI_Count']/16848906)*100
l2_6['%UMI']=(l2_6['UMI_Count']/16071069)*100
l3_2['%UMI']=(l3_2['UMI_Count']/25457517)*100
l3_3['%UMI']=(l3_3['UMI_Count']/15639047)*100
l3_4['%UMI']=(l3_4['UMI_Count']/20028038)*100
l3_5['%UMI']=(l3_5['UMI_Count']/18347023)*100

l2_2_s=l2_2.sort_values(by='%UMI',ascending=False)
l2_2=l2_2_s.reset_index()
l2_2['rank']=l2_2.index
l2_3_s=l2_3.sort_values(by='%UMI',ascending=False)
l2_3=l2_3_s.reset_index()
l2_3['rank']=l2_3.index
l2_4_s=l2_4.sort_values(by='%UMI',ascending=False)
l2_4=l2_4_s.reset_index()
l2_4['rank']=l2_4.index
l2_6_s=l2_6.sort_values(by='%UMI',ascending=False)
l2_6=l2_6_s.reset_index()
l2_6['rank']=l2_6.index

l3_2_s=l3_2.sort_values(by='%UMI',ascending=False)
l3_2=l3_2_s.reset_index()
l3_2['rank']=l3_2.index
l3_3_s=l3_3.sort_values(by='%UMI',ascending=False)
l3_3=l3_3_s.reset_index()
l3_3['rank']=l3_3.index
l3_4_s=l3_4.sort_values(by='%UMI',ascending=False)
l3_4=l3_4_s.reset_index()
l3_4['rank']=l3_4.index
l3_5_s=l3_5.sort_values(by='%UMI',ascending=False)
l3_5=l3_5_s.reset_index()
l3_5['rank']=l3_5.index

df = pd.DataFrame(index=range(len(l2_4)-len(l2_2)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l2_2 = pd.concat([l2_2, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l2_4)-len(l2_3)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l2_3 = pd.concat([l2_3, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l2_4)-len(l2_6)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l2_6 = pd.concat([l2_6, df], ignore_index = True)

df = pd.DataFrame(index=range(len(l2_4)-len(l3_2)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_2 = pd.concat([l3_2, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l2_4)-len(l3_3)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_3 = pd.concat([l3_3, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l2_4)-len(l3_4)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_4 = pd.concat([l3_4, df], ignore_index = True)
df = pd.DataFrame(index=range(len(l2_4)-len(l3_5)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
l3_5 = pd.concat([l3_5, df], ignore_index = True)

plt.plot(np.log10(l2_2['rank']),l2_2['%UMI'],label = "Lipofectamine2000",color = '#0077BB',linewidth=1.5)
plt.plot(np.log10(l3_2['rank']),l3_2['%UMI'],label = "Lipofectamine3000",color = '#882255',linewidth=1.5)
plt.plot(np.log10(l2_3['rank']),l2_3['%UMI'],color = '#0077BB',linewidth=1.5,alpha=0.8)
plt.plot(np.log10(l3_3['rank']),l3_3['%UMI'],color = '#882255',linewidth=1.5,alpha=0.8)
plt.plot(np.log10(l2_4['rank']),l2_4['%UMI'],color = '#0077BB',linewidth=1.5,alpha=0.5)
plt.plot(np.log10(l3_4['rank']),l3_4['%UMI'],color = '#882255',linewidth=1.5,alpha=0.5)
plt.plot(np.log10(l2_6['rank']),l2_6['%UMI'],color = '#0077BB',linewidth=1.5,alpha=0.3)
plt.plot(np.log10(l3_5['rank']),l3_5['%UMI'],color = '#882255',linewidth=1.5,alpha=0.3)



plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Lipofectamine Version")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
#plt.axis([0, 7, 0.95, 1000000])
plt.legend(loc='upper right')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/rep_lipofectamine_comparison.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
#plate size
#cell line comparison (lipofectamine3000, hekgt vs. hek293t)
hek_enva=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/plate_size/15_completecounts.tsv',delimiter='\t')
l3_enva=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/plate_size/384_completecounts.tsv',delimiter='\t')

print(hek_enva['UMI_Count'].sum())
print(l3_enva['UMI_Count'].sum())

print(len(hek_enva))
print(len(l3_enva))

hek_enva['%UMI']=(hek_enva['UMI_Count']/110336495)*100
l3_enva['%UMI']=(l3_enva['UMI_Count']/111654453)*100

hek_enva_s=hek_enva.sort_values(by='%UMI',ascending=False)
hek_enva=hek_enva_s.reset_index()
hek_enva['rank']=hek_enva.index

l3_enva_s=l3_enva.sort_values(by='%UMI',ascending=False)
l3_enva=l3_enva_s.reset_index()
l3_enva['rank']=l3_enva.index


df = pd.DataFrame(index=range(len(l3_enva)-len(hek_enva)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
hek_enva = pd.concat([hek_enva, df], ignore_index = True)


plt.plot(np.log10(hek_enva['rank']),hek_enva['%UMI'],label = "384 well",color = '#0077BB',linewidth=1.5)
plt.plot(np.log10(l3_enva['rank']),l3_enva['%UMI'],label = "15 cm",color = '#882255',linewidth=1.5)


plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Cell Line")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
#plt.axis([0, 7, 0.95, 1000000])
plt.legend(loc='upper right')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/plate_size.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
#previous dataset vs replicates
#processing saunders dataset:
saunders_temp=pd.read_table('/Users/maddieurbanek/Desktop/pseudotypedcounts.csv',delimiter=',')
print(saunders_temp)
#Finish processing Saunders dataset
saunders = saunders_temp.loc[saunders_temp.index.repeat(saunders_temp['n_count'])].reset_index(drop=True)
saunders['barcode'] = saunders.index
saunders.drop(['n_count'],axis=1)
umis = saunders.loc[saunders.index.repeat(saunders['n_unique_umibc_pairs'])].reset_index(drop=True)
saunders=umis.drop(['n_count','n_unique_umibc_pairs'], axis=1)
saunders=pd.DataFrame(saunders)
saunders=pd.DataFrame(saunders)
saunders=saunders['barcode'].value_counts().reset_index()
saunders.columns=['barcode','UMI_Count']
#saunders.to_csv('/Users/maddieurbanek/Desktop/saunders.csv')

In [ ]:
#previous dataset vs replicates

rep1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/gt_completecounts.tsv',delimiter='\t')
rep2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/envb.tsv',delimiter='\t')
rep3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/enva.tsv',delimiter='\t')
#rep4=pd.read_table('/Users/maddieurbanek/Desktop/h2b_completecounts.tsv',delimiter='\t')
rep5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cvs_viral/cvs_completecounts.tsv',delimiter='\t')
chen=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/zhang/zhangcompletecounts.csv',delimiter=',')
clark=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/clark/clarkcounts.tsv',delimiter='\t')
saunders=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/saunders/saunders.csv',delimiter=',')

rep3 = pd.DataFrame(rep3)
rep3 = rep3.reset_index()
test_1 = rep3.loc[rep3.index.repeat(rep3['UMI_Count'])].reset_index(drop=True)
test_1=test_1['barcode'].value_counts()
test_1=pd.DataFrame(test_1)
rep3 = test_1.reset_index()
rep3.columns=['barcode','UMI_Count']

print(rep1['UMI_Count'].sum())
print(rep2['UMI_Count'].sum())
print(rep3['UMI_Count'].sum())
#print(rep4['UMI_Count'].sum())
print(rep5['UMI_Count'].sum())
print(chen['UMI_Count'].sum())
print(clark['UMI_Count'].sum())
print(saunders['UMI_Count'].sum())

print(len(rep1))
print(len(rep2))
print(len(rep3))
#print(len(rep4))
print(len(rep5))
print(len(chen))
print(len(clark))
print(len(saunders))
rep1['%UMI']=(rep1['UMI_Count']/98288058)*100
rep2['%UMI']=(rep2['UMI_Count']/1339366)*100
rep3['%UMI']=(rep3['UMI_Count']/4830613)*100
#rep4['%UMI']=(rep4['UMI_Count']/9795086)*100
rep5['%UMI']=(rep5['UMI_Count']/61765229)*100
chen['%UMI']=(chen['UMI_Count']/399833)*100
clark['%UMI']=(clark['UMI_Count']/16446027)*100
saunders['%UMI']=(saunders['UMI_Count']/8239362.00255729)*100

rep1_s=rep1.sort_values(by='%UMI',ascending=False)
rep1=rep1_s.reset_index()
rep1['rank']=rep1.index
rep2_s=rep2.sort_values(by='%UMI',ascending=False)
rep2=rep2_s.reset_index()
rep2['rank']=rep2.index
rep3_s=rep3.sort_values(by='%UMI',ascending=False)
rep3=rep3_s.reset_index()
rep3['rank']=rep3.index
rep4_s=rep4.sort_values(by='%UMI',ascending=False)
#rep4=rep4_s.reset_index()
#rep4['rank']=rep4.index
rep5_s=rep5.sort_values(by='%UMI',ascending=False)
rep5=rep5_s.reset_index()
rep5['rank']=rep5.index

chen_s=chen.sort_values(by='%UMI',ascending=False)
chen=chen_s.reset_index()
chen['rank']=chen.index
clark_s=clark.sort_values(by='%UMI',ascending=False)
clark=clark_s.reset_index()
clark['rank']=clark.index
saunders_s=saunders.sort_values(by='%UMI',ascending=False)
saunders=saunders_s.reset_index()
saunders['rank']=saunders.index


df = pd.DataFrame(index=range(len(rep4)-len(rep1)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
rep1 = pd.concat([rep1, df], ignore_index = True)
df = pd.DataFrame(index=range(len(rep4)-len(rep2)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
rep2 = pd.concat([rep2, df], ignore_index = True)
df = pd.DataFrame(index=range(len(rep4)-len(rep3)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
rep3 = pd.concat([rep3, df], ignore_index = True)
df = pd.DataFrame(index=range(len(rep4)-len(rep5)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
rep5 = pd.concat([rep5, df], ignore_index = True)
df = pd.DataFrame(index=range(len(rep4)-len(chen)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
chen = pd.concat([chen, df], ignore_index = True)
df = pd.DataFrame(index=range(len(rep4)-len(clark)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
clark = pd.concat([clark, df], ignore_index = True)
df = pd.DataFrame(index=range(len(rep4)-len(saunders)),columns=range(1))
df.fillna(-100, inplace=True)
df.columns=['%UMI']
saunders = pd.concat([saunders, df], ignore_index = True)

plt.plot(np.log10(rep1['rank']),rep1['%UMI'],label = "This Study",color = '#882255',linewidth=1.5)
plt.plot(np.log10(rep2['rank']),rep2['%UMI'],color = '#882255',linewidth=1.5,alpha=0.8)
plt.plot(np.log10(rep3['rank']),rep3['%UMI'],color = '#882255',linewidth=1.5,alpha=0.6)
#plt.plot(np.log10(rep4['rank']),rep4['%UMI'],color = '#882255',linewidth=1.5,alpha=0.4)
plt.plot(np.log10(rep5['rank']),rep5['%UMI'],color = '#882255',linewidth=1.5,alpha=0.2)
plt.plot(np.log10(chen['rank']),chen['%UMI'],label = "Zhang 2024",color = '#88CCEE',linewidth=1.5)
plt.plot(np.log10(saunders['rank']),saunders['%UMI'],label = "Saunders 2022",color = '#332288',linewidth=1.5)
plt.plot(np.log10(clark['rank']),clark['%UMI'],label = "Clark 2021",color = '#117733',linewidth=1.5)


plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("%UMI")
plt.title("Previously Published Datasets")
#plt.yscale('log')
#plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
plt.axis([0, 7, 0.95, 1000000])
plt.legend(loc='upper right')
plt.draw()
#plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/publication_comparison.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
print(saunders)

#plt.plot((rep1['rank']),np.log10(rep1['UMI_Count']),label = "This Study",color = '#882255',linewidth=1.5)
#plt.plot((rep2['rank']),np.log10(rep2['UMI_Count']),color = '#882255',linewidth=1.5,alpha=0.8)
#plt.plot((rep3['rank']),np.log10(rep3['UMI_Count']),color = '#882255',linewidth=1.5,alpha=0.6)
#plt.plot((rep4['rank']),np.log10(rep4['UMI_Count']),color = '#882255',linewidth=1.5,alpha=0.4)
#plt.plot((rep5['rank']),np.log10(rep5['UMI_Count']),color = '#882255',linewidth=1.5,alpha=0.2)
plt.plot((chen['rank']),np.log10(chen['UMI_Count']),label = "Zhang 2024",color = '#88CCEE',linewidth=1.5)
plt.plot((saunders['rank']),np.log10(saunders['UMI_Count']),label = "Saunders 2022",color = '#332288',linewidth=1.5)
plt.plot((clark['rank']),np.log10(clark['UMI_Count']),label = "Clark 2021",color = '#117733',linewidth=1.5)
plt.plot((rep1['rank']),np.log10(rep1['UMI_Count']),label = "This Study",color = '#882255',linewidth=1.5)

plt.xlabel("Log10(Barcode Rank)")
plt.ylabel("")
plt.title("Previously Published Datasets")
#plt.yscale('log')
plt.legend(loc='upper right')
# Adding legend, which helps us recognize the curve according to it's color
plt.axis([0, 20000, 0, 5])
plt.legend(loc='upper right')
plt.draw()
#plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/non-logged_knee/publication_comparison.pdf",bbox_inches='tight', format='pdf')
plt.show()

In [ ]:
#Plot barcode collision w/ error

In [ ]:
#cvs=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/gt_completecounts.tsv',delimiter='\t')
#cvs=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/envb.tsv',delimiter='\t')
#rep3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/enva.tsv',delimiter='\t')
rep3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/enva.tsv',delimiter='\t')
rep3 = pd.DataFrame(rep3)
rep3 = rep3.reset_index()
test_1 = rep3.loc[rep3.index.repeat(rep3['UMI_Count'])].reset_index(drop=True)
test_1=test_1['barcode'].value_counts()
test_1=pd.DataFrame(test_1)
cvs = test_1.reset_index()
cvs.columns=['barcode','UMI_Count']
#cvs=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cvs_viral/cvs_completecounts.tsv',delimiter='\t')
#cvs=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/zhang/zhangcompletecounts.csv',delimiter=',')
cvs = cvs.loc[cvs.index.repeat(cvs['UMI_Count'])].reset_index(drop=True)
print(cvs)

In [ ]:
int_list = [130000000000000]
i=100000
for n in range(0,50):
    data=(np.random.choice(cvs['barcode'],i,replace=True))
    data=pd.DataFrame(data)
    dup = data.duplicated(keep=first).sum()
    int_list.append(dup)
int_list = int_list[1:]
int_list=pd.DataFrame(int_list)
percentcells = int_list / i
int_list = pd.concat([int_list, percentcells], axis=1, ignore_index=True)
int_list.columns=['collisioncells','percentcells']
print(len(int_list))
int_list.to_csv('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep1_5.txt', index = False, sep = '\t')

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep1_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep1_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep1_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep1_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep1_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
rep1=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep2_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep2_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep2_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep2_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep2_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
rep2=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep3_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep3_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep3_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep3_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep3_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
rep3=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep4_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep4_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep4_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep4_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/rep4_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
rep4=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/saunders_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/saunders_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/saunders_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/saunders_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/saunders_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
saunders=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/clark_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/clark_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/clark_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/clark_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/clark_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
clark=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/chen_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/chen_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/chen_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/chen_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/prev_data/keep_true/chen_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
chen=new.copy(deep=True)

In [ ]:
sim_1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/new_transfection/l3k.5_1.txt')
sim_2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/new_transfection/l3k.5_2.txt')
sim_3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/new_transfection/l3k.5_3.txt')
sim_4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/new_transfection/l3k.5_4.txt')
sim_5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/collision/new_transfection/l3k.5_5.txt')

threemeans = [130000000000000]
threemean=sim_1['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_2['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_3['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_4['percentcells'].mean()
threemeans.append(threemean)
threemean=sim_5['percentcells'].mean()
threemeans.append(threemean)

threemeans = threemeans[1:]

chencis = [130000000000000]
chenci=st.t.interval(confidence=0.95, df=len(sim_1['percentcells'])-1, loc=np.mean(sim_1['percentcells']), scale=st.sem(sim_1['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_2['percentcells'])-1, loc=np.mean(sim_2['percentcells']), scale=st.sem(sim_2['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_3['percentcells'])-1, loc=np.mean(sim_3['percentcells']), scale=st.sem(sim_3['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_4['percentcells'])-1, loc=np.mean(sim_4['percentcells']), scale=st.sem(sim_4['percentcells'])) 
chencis.append(chenci)
chenci=st.t.interval(confidence=0.95, df=len(sim_5['percentcells'])-1, loc=np.mean(sim_5['percentcells']), scale=st.sem(sim_5['percentcells'])) 
chencis.append(chenci)

chencis = chencis[1:]

new=[1,2,3,4,5]
new=pd.DataFrame(new)
chencis=pd.DataFrame(chencis)
threemeans=pd.DataFrame(threemeans)
new = pd.concat([new, threemeans], axis=1, ignore_index=True)
new = pd.concat([new, chencis], axis=1, ignore_index=True)
new.columns=['x','mean','higher','lower']
l3k_4=new.copy(deep=True)

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
sns.lineplot(data=rep1, x="x", y="mean", ci='sd', color='black',linewidth = 1.5)
sns.lineplot(data=rep2, x="x", y="mean", ci='sd',color='black', linewidth = 1.5,alpha=0.8)
sns.lineplot(data=rep3, x="x", y="mean", ci='sd', color='black',linewidth = 1.5,alpha=0.6)
sns.lineplot(data=rep4, x="x", y="mean", ci='sd',color='black', linewidth = 1.5,alpha=0.4)

sns.lineplot(data=chen, x="x", y="mean", ci='sd', color='blue',linewidth = 1.5)
sns.lineplot(data=saunders, x="x", y="mean", ci='sd',color='green', linewidth = 1.5)
sns.lineplot(data=clark, x="x", y="mean", ci='sd', color='purple',linewidth = 1.5)
#sns.lineplot(data=l3k_4, x="x", y="mean", ci='sd',color='blue', linewidth = 1.5)

#sns.lineplot(data=saunders, x="x", y="mean", ci='sd',color='indigo', linewidth = 1.5,label='saunders')
#sns.lineplot(data=clark, x="x", y="mean", errorbar='sd',color='green', linewidth = 1.5,label='clark')
#sns.lineplot(data=chen, x="x", y="mean", ci='sd',color='skyblue', linewidth = 1.5,label='chen')

#sns.lineplot(data=sad, x="x", y="mean", errorbar='sd',color='green', linewidth = 1.5,label='sad')
#sns.lineplot(data=cvs, x="x", y="mean", errorbar='sd',color='skyblue', linewidth = 1.5,label='cvs')
ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/prev_pub.svg")
plt.show()

In [ ]:
#Plot read simulations

In [ ]:
#previous dataset vs replicates
#processing saunders dataset:
rep1=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/gt_completecounts.tsv',delimiter='\t')
rep2=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/envb.tsv',delimiter='\t')
rep3=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/enva.tsv',delimiter='\t')
#rep4=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/optimized_replicates/h2b.csv',delimiter=',')
rep5=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/cvs_viral/cvs_completecounts.tsv',delimiter='\t')
chen=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/zhang/zhangcompletecounts.csv',delimiter=',')
clark=pd.read_table('/Users/maddieurbanek/Downloads/Source/raw_data/previous_studies/clark/clarkcounts.tsv',delimiter='\t')
saunders=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/saunders/saunders.csv',delimiter=',')

rep3 = pd.DataFrame(rep3)
rep3 = rep3.reset_index()
test_1 = rep3.loc[rep3.index.repeat(rep3['UMI_Count'])].reset_index(drop=True)
test_1=test_1['barcode'].value_counts()
test_1=pd.DataFrame(test_1)
rep3 = test_1.reset_index()
rep3.columns=['barcode','UMI_Count']
print(rep3.head)



In [ ]:
#Bootstrapping 10^1-6 and save datsets
chen=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/previous_studies/zhang/zhangcompletecounts.csv',delimiter=',')

chen['barcode'] = chen.index
#clark['barcode'] = clark.index
#saunders['barcode']= saunders.index
#rep1['barcode']= rep1.index
#rep2['barcode']= rep2.index
#rep3['barcode']= rep3.index
#rep5['barcode']= rep5.index

chen = chen.loc[chen.index.repeat(chen['UMI_Count'])].reset_index(drop=True)
#clark = clark.loc[clark.index.repeat(clark['UMI_Count'])].reset_index(drop=True)
#saunders = saunders.loc[saunders.index.repeat(saunders['UMI_Count'])].reset_index(drop=True)
#rep1 = rep1.loc[rep1.index.repeat(rep1['UMI_Count'])].reset_index(drop=True)
#rep2 = rep2.loc[rep2.index.repeat(rep2['UMI_Count'])].reset_index(drop=True)
#rep3 = rep3.loc[rep3.index.repeat(rep3['UMI_Count'])].reset_index(drop=True)
#rep5 = rep5.loc[rep5.index.repeat(rep5['UMI_Count'])].reset_index(drop=True)

chenbootcounts = pd.DataFrame()
i=1000
for n in range(0,100):
    data=(np.random.choice(chen['barcode'],i,replace=True))
    data=pd.Series(data).value_counts()
    data = pd.DataFrame(data)
    data.columns=['count']
    data=data.reset_index()
    chenbootcounts=pd.concat([chenbootcounts,data['count']],ignore_index=True,axis=1)

chenbootcounts.to_csv('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_chen.csv')
print(chenbootcounts)

In [ ]:
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_chen.csv',delimiter=',')
chenmeans = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
chenmeans.append(chenmean)
chenmeans = chenmeans[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
chenmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
chenmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
chenmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
chenmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
chenmeans.append(chenmean)
print(chenmeans)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_chen.csv',delimiter=',')
chendevs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
chendevs.append(chendev)
chendevs = chendevs[1:]
print(chendevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
chendevs.append(chendev)
print(chendevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
chendevs.append(chendev)
print(chendevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
chendevs.append(chendev)
print(chendevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
chendevs.append(chendev)
print(chendevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_chen.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
chendevs.append(chendev)
print(chendevs)

#
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_clark.csv',delimiter=',')
clarkmeans = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
clarkmeans.append(chenmean)
clarkmeans = clarkmeans[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
clarkmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
clarkmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
clarkmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
clarkmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
clarkmeans.append(chenmean)


chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_clark.csv',delimiter=',')
clarkdevs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
clarkdevs.append(chendev)
clarkdevs = clarkdevs[1:]
print(clarkdevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
clarkdevs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
clarkdevs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
clarkdevs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
clarkdevs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_clark.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
clarkdevs.append(chendev)

#
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_saunders.csv',delimiter=',')
saundersmeans = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
saundersmeans.append(chenmean)
saundersmeans = saundersmeans[1:]
print(saundersmeans)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
saundersmeans.append(chenmean)
print(saundersmeans)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
saundersmeans.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
saundersmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
saundersmeans.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
saundersmeans.append(chenmean)


chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_saunders.csv',delimiter=',')
saundersdevs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
saundersdevs.append(chendev)
saundersdevs = saundersdevs[1:]
print(saundersdevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
saundersdevs.append(chendev)
print(saundersdevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
saundersdevs.append(chendev)
print(saundersdevs)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
saundersdevs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
saundersdevs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_saunders.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
saundersdevs.append(chendev)




#
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep1.csv',delimiter=',')
rep1means = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep1means.append(chenmean)
rep1means = rep1means[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep1means.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep1means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep1means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep1means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep1means.append(chenmean)





chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep1.csv',delimiter=',')
rep1devs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep1devs.append(chendev)
rep1devs = rep1devs[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep1devs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep1devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep1devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep1devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep1.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep1devs.append(chendev)



#
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep2.csv',delimiter=',')
rep2means = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep2means.append(chenmean)
rep2means = rep2means[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep2means.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep2means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep2means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep2means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep2means.append(chenmean)


chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep2.csv',delimiter=',')
rep2devs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep2devs.append(chendev)
rep2devs = rep2devs[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep2devs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep2devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep2devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep2devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep2.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep2devs.append(chendev)
#
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep3.csv',delimiter=',')
rep3means = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep3means.append(chenmean)
rep3means = rep3means[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep3means.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep3means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep3means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep3means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep3means.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep3.csv',delimiter=',')
rep3devs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep3devs.append(chendev)
rep3devs = rep3devs[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep3devs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep3devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep3devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep3devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep3.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep3devs.append(chendev)


chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep5.csv',delimiter=',')
rep5means = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep5means.append(chenmean)
rep5means = rep5means[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep5means.append(chenmean)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep5means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep5means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep5means.append(chenmean)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chenmean=df.mean()
rep5means.append(chenmean)


chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/1_rep5.csv',delimiter=',')
rep5devs = [130000000000000]
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep5devs.append(chendev)
rep5devs = rep5devs[1:]

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/2_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep5devs.append(chendev)

chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/3_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep5devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/4_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep5devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/5_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep5devs.append(chendev)
chen = pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes/6_rep5.csv',delimiter=',')
chen=chen.iloc[: , 1:]
df=chen.count()
chendev=df.std()
rep5devs.append(chendev)

#
reads=[10,100,1000,10000,100000,1000000]
reads=pd.DataFrame(reads)
chenmeans=pd.DataFrame(chenmeans)
clarkmeans=pd.DataFrame(clarkmeans)
saundersmeans=pd.DataFrame(saundersmeans)
rep1means=pd.DataFrame(rep1means)
rep2means=pd.DataFrame(rep2means)
rep3means=pd.DataFrame(rep3means)
rep5means=pd.DataFrame(rep5means)

chendevs=pd.DataFrame(chendevs)
clarkdevs=pd.DataFrame(clarkdevs)
saundersdevs=pd.DataFrame(saundersdevs)
rep1devs=pd.DataFrame(rep1devs)
rep2devs=pd.DataFrame(rep2devs)
rep3devs=pd.DataFrame(rep3devs)
rep5devs=pd.DataFrame(rep5devs)

reads.columns=['x']
reads = pd.concat([reads, chenmeans], axis=1, ignore_index=True)
reads = pd.concat([reads, clarkmeans], axis=1, ignore_index=True)
reads = pd.concat([reads, saundersmeans], axis=1, ignore_index=True)
reads = pd.concat([reads, rep1means], axis=1, ignore_index=True)
reads = pd.concat([reads, rep2means], axis=1, ignore_index=True)
reads = pd.concat([reads, rep3means], axis=1, ignore_index=True)
reads = pd.concat([reads, rep5means], axis=1, ignore_index=True)
ideal=[10,100,1000,10000,100000,1000000]
ideal=pd.DataFrame(ideal)
idealdev=[0,0,0,0,0,0]
idealdev=pd.DataFrame(idealdev)
reads = pd.concat([reads, ideal], axis=1, ignore_index=True)

reads = pd.concat([reads, chendevs], axis=1, ignore_index=True)
reads = pd.concat([reads, clarkdevs], axis=1, ignore_index=True)
reads = pd.concat([reads, saundersdevs], axis=1, ignore_index=True)
reads = pd.concat([reads, rep1devs], axis=1, ignore_index=True)
reads = pd.concat([reads, rep2devs], axis=1, ignore_index=True)
reads = pd.concat([reads, rep3devs], axis=1, ignore_index=True)
reads = pd.concat([reads, rep5devs], axis=1, ignore_index=True)
reads = pd.concat([reads, idealdev], axis=1, ignore_index=True)

reads.columns=['x','chen','clark','saunders','rep1','rep2','rep3','rep5', 'ideal', 'chendev','clarkdev','saundersdev','rep1dev','rep2dev','rep3dev','rep5dev','idealdev']

reads.to_csv('/Users/maddieurbanek/Desktop/rabies_project/data/unique_barcodes_vs_reads.csv')


In [ ]:
#Plot unique barcode per reads comparison
reads=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/diversity_datasets/unique_barcodes_vs_reads.csv',delimiter=',')
plt.errorbar(reads['x'],reads['clark'], yerr=reads['clarkdev'], color = '#117733',label="Clark et al 2021")
plt.errorbar(reads['x'],reads['saunders'], yerr=reads['saundersdev'], color = '#332288',label="Saunders et al 2022")
plt.errorbar(reads['x'],reads['chen'], yerr=reads['chendev'], color = '#88CCEE',label="Zhang et al 2023")

plt.errorbar(reads['x'],reads['rep1'], yerr=reads['rep1dev'], color = '#AA4499',label="Rep1",linewidth=1)
plt.errorbar(reads['x'],reads['rep2'], yerr=reads['rep2dev'], color = '#AA4499',label="Rep2",alpha=0.7,linewidth=1)
plt.errorbar(reads['x'],reads['rep3'], yerr=reads['rep3dev'], color = '#AA4499',label="Rep3",alpha=0.5,linewidth=1)
plt.errorbar(reads['x'],reads['rep5'], yerr=reads['rep5dev'], color = '#AA4499',label="Rep5",alpha=0.3,linewidth=1)
plt.errorbar(reads['x'],reads['ideal'], yerr=reads['idealdev'], color = 'black',label="Ideal Barcode System", fmt=":",linewidth=1)


plt.fill_between(reads['x'], reads['chen']-reads['chendev'], reads['chen']+reads['chendev'],color='#88CCEE',alpha=0.3,linewidth=1)
plt.fill_between(reads['x'], reads['clark']-reads['clarkdev'], reads['clark']+reads['clarkdev'],color='#117733',alpha=0.3,linewidth=1)
plt.fill_between(reads['x'], reads['saunders']-reads['saundersdev'], reads['saunders']+reads['saundersdev'],color='#332288',alpha=0.3,linewidth=1)
plt.fill_between(reads['x'], reads['rep1']-reads['rep1dev'], reads['rep1']+reads['rep1dev'],color='#AA4499',alpha=0.3,linewidth=1)
plt.fill_between(reads['x'], reads['rep2']-reads['rep2dev'], reads['rep2']+reads['rep2dev'],color='#AA4499',alpha=0.1,linewidth=1)
plt.fill_between(reads['x'], reads['rep3']-reads['rep3dev'], reads['rep3']+reads['rep3dev'],color='#AA4499',alpha=0.1,linewidth=1)
plt.fill_between(reads['x'], reads['rep5']-reads['rep5dev'], reads['rep5']+reads['rep5dev'],color='#AA4499',alpha=0.1,linewidth=1)


plt.ylabel("# Unique Barcodes")
plt.xlabel("Simulated Reads")
plt.title("Unique Barcodes per Simulated Reads")
plt.legend()

#plt.axis([5, 6.5, 1000, 1000000])


#import matplotlib as mpl
#mpl.rcParams['pdf.fonttype'] = 42
#mpl.rcParams['ps.fonttype'] = 42
#plt.draw()
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/1-3_diversity/unique_barcodes_per_simulated_reads.pdf', bbox_inches='tight', format='pdf')
plt.show()


In [ ]:
FIG 7: Empty droplet analysis

In [ ]:
NUC ONLY

In [ ]:
#Pull empty droplets for nuc datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

In [ ]:
#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

In [ ]:
#Get all cell-matched barcodes, w/o UMI thresholding
actualbarcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

In [ ]:
#Plot total transcripts distribution
print(np.log10(summed['UMI_Count'].min()))
print(np.log10(summed['UMI_Count'].max()))
print('Number of nuc:')
print(summed['CBC'].nunique())
print('Mean of total transcripts per cell/nuc')
print(summed['UMI_Count'].mean())
print('Median of total transcripts per cell/nuc')
print(summed['UMI_Count'].median())
print('sem of total transcripts per cell/nuc')
print(summed['UMI_Count'].sem())

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()
print(np.log10(summed_empty['UMI_Count'].min()))
print(np.log10(summed_empty['UMI_Count'].max()))
print('Number of empty droplets:')
print(summed_empty['CBC'].nunique())
print('Mean of total transcripts per empty droplet')
print(summed_empty['UMI_Count'].mean())
print('Median of total transcripts per empty droplet')
print(summed_empty['UMI_Count'].median())
print('sem of total transcripts per empty droplet')
print(summed_empty['UMI_Count'].sem())

summed_empty.to_csv('/Users/maddieurbanek/Desktop/summed_empty.csv')
summed.to_csv('/Users/maddieurbanek/Desktop/summed.csv')

plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist((np.log10(summed['UMI_Count'])),bins=np.arange(0, 3.461198288622493 + .01, .05),alpha=0.5,color='orchid',label='Real Cells/Nuc')
plt.hist((np.log10(summed_empty['UMI_Count'])),bins=np.arange(0, 3.461198288622493 + .01, .05),alpha=0.3,color='green',label='Empty Droplets')

plt.xlim([-.1, 5])
plt.legend(loc='upper right')
plt.xlabel('Log10(Total Barcode Transcripts per Nuclei)')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/nuc_total_transcripts.svg', bbox_inches='tight', format='svg')
plt.show()

print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
from scipy.stats import ranksums

drop_dist = summed_empty['UMI_Count']
cell_dist = summed['UMI_Count']

print('Wilcoxon Ranked Sum Results:')
print(ranksums(drop_dist, cell_dist))

In [ ]:
#Plot combo
print((nuc_barcodes['UMI_Count'].min()))
print((nuc_barcodes['UMI_Count'].max()))
print('Number of nuc:barcode combos:')
print(len(nuc_barcodes))
print('Mean of unique barcode transcripts per cell/nuc')
print(nuc_barcodes['UMI_Count'].mean())
print('Median of unique barcode transcripts per cell/nuc')
print(nuc_barcodes['UMI_Count'].median())
print('sem of total transcripts per cell/nuc')
print(nuc_barcodes['UMI_Count'].sem())

print((empty_barcodes['UMI_Count'].min()))
print((empty_barcodes['UMI_Count'].max()))
print('Number of droplet:barcode combos:')
print(len(empty_barcodes))
print('Mean of unique barcode transcripts per empty droplet')
print(empty_barcodes['UMI_Count'].mean())
print('Median of unique barcode transcripts per empty droplet')
print(empty_barcodes['UMI_Count'].median())
print('sem of total transcripts per empty droplet')
print(empty_barcodes['UMI_Count'].sem())

plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist(((nuc_barcodes['UMI_Count'])),bins=np.arange(1, 862 + 100, 10),alpha=0.5,color='orchid',label='Real Nuc')
plt.hist(((empty_barcodes['UMI_Count'])),bins=np.arange(1, 240 + 100, 10),alpha=0.3,color='green',label='Empty Droplets')


#plt.xlim([-.1, 5])
plt.legend(loc='upper right')
plt.xlabel('Barcode Transcripts per Cell:Barcode Combination')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/nuc_combo_transcripts.svg', bbox_inches='tight', format='svg')
plt.show()

print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
from scipy.stats import ranksums

drop_dist = empty_barcodes['UMI_Count']
cell_dist = nuc_barcodes['UMI_Count']

print('Wilcoxon Ranked Sum Results:')
print(ranksums(drop_dist, cell_dist))

In [ ]:
#Zoomed
plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist(((nuc_barcodes['UMI_Count'])),bins=np.arange(1, 24169 + 100, 3),alpha=0.5,color='orchid',label='Real Nuc')
plt.hist(((empty_barcodes['UMI_Count'])),bins=np.arange(1, 533 + 100, 3),alpha=0.3,color='green',label='Empty Droplets')

plt.xlim([-10, 200])
plt.legend(loc='upper right')
plt.xlabel('Log10(Barcode Transcripts per Cell:Barcode Combination)')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/nuc_zoomed_combo_transcripts.svg', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
#Calculate % of UMIs each barcode takes up per cell

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

combo_actual=new_actual.copy(deep=True)
combo_empty=new_empty.copy(deep=True)
print(len(new_actual))
new_actual=new_actual.drop_duplicates('CBC').sort_index()
print(len(new_actual))
new_empty=new_empty.drop_duplicates('CBC').sort_index()


In [ ]:
#Plot unique number of barcodes per cell/droplet
print(new_actual['logged_unique'].min())
print(new_actual['logged_unique'].max())
print('Number of nuc:')
print(new_actual['CBC'].nunique())
print('Mean of unique barcodes for cell/nuc')
print(new_actual['logged_unique'].mean())
print('Median of unique barcodes for cell/nuc')
print(new_actual['logged_unique'].median())
print('SEM of unique barcodes for cell/nuc')
print(new_actual['logged_unique'].sem())

print('Mean of unique barcodes for cell/nuc')
print(new_actual['barcode_y'].mean())
print('Median of unique barcodes for cell/nuc')
print(new_actual['barcode_y'].median())
print('SEM of unique barcodes for cell/nuc')
print(new_actual['barcode_y'].sem())

print((new_empty['logged_unique'].min()))
print(new_empty['logged_unique'].max())
print('Number of droplets:')
print(new_empty['CBC'].nunique())
print('Mean of unique barcodes for empty droplet')
print(new_empty['logged_unique'].mean())
print('Median of unique barcodes for empty droplet')
print(new_empty['logged_unique'].median())
print('SEM of unique barcodes for empty droplet')
print(new_empty['logged_unique'].sem())

print('Mean of unique barcodes for empty droplet')
print(new_empty['barcode_y'].mean())
print('Median of unique barcodes for empty droplet')
print(new_empty['barcode_y'].median())
print('SEM of empty droplets for cell/nuc')
print(new_empty['barcode_y'].sem())

new_empty.to_csv('/Users/maddieurbanek/Desktop/new_empty.csv')
new_actual.to_csv('/Users/maddieurbanek/Desktop/new_actual.csv')

plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist(((new_actual['logged_unique'])),bins=np.arange(0, 3.209783014848515 + .01, .05),alpha=0.5,color='orchid',label='Real Nuc')
plt.hist(((new_empty['logged_unique'])),bins=np.arange(0, 3.209783014848515 + .01, .05),alpha=0.3,color='green',label='Empty Droplets')

plt.legend(loc='upper right')
plt.xlabel('Log10(Unique Barcodes)')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/nuc_unique_barcode.svg', bbox_inches='tight', format='svg')
plt.show()

print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
from scipy.stats import ranksums

drop_dist = new_empty['logged_unique']
cell_dist = new_actual['logged_unique']

print('Wilcoxon Ranked Sum Results:')
print(ranksums(drop_dist, cell_dist))

In [ ]:
new_empty['logged_combo']=np.log10(new_empty['UMI_Count_x'])
new_actual['logged_combo']=np.log10(new_actual['UMI_Count_x'])
new_actual['datasetid']='actual'
new_empty['datasetid']='empty'
barplot=pd.concat([new_actual, new_empty], axis=0)

In [ ]:
combos = barplot[['logged_combo', 'datasetid']].copy()
combos[['origin']]='combo'
print(combos)

subset_barplot=barplot.copy(deep=True)
subset_barplot=subset_barplot.drop_duplicates(subset=['CBC'])

totals = subset_barplot[['logged_total', 'datasetid']].copy()
totals.columns=['logged_combo','datasetid']
totals[['origin']]='totals'
print(totals)

uniques = subset_barplot[['logged_unique', 'datasetid']].copy()
uniques.columns=['logged_combo','datasetid']
uniques[['origin']]='unique'
print(uniques)

barplot_input=pd.concat([combos, totals, uniques], axis=0)
print(barplot_input)

In [ ]:
#Summary stats bar plot
ax=sns.barplot(data=barplot_input, x="origin", y="logged_combo", hue="datasetid",errorbar="sd",zorder=10)
#ax=sns.swarmplot(x="origin", y="logged_combo", data=barplot_input, color="0", alpha=.1)
#ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/nuc_pre_stats.svg")
plt.show()


print('Welchs T-test to compare means for cell:barcode combos:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(new_actual['logged_combo'], new_empty['logged_combo'],equal_var=False))

print('Welchs T-test to compare means for total transcripts:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(new_actual['logged_total'], new_empty['logged_total'],equal_var=False))

print('Welchs T-test to compare means for unique barcodes:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(new_actual['logged_unique'], new_empty['logged_unique'],equal_var=False))

In [ ]:
#Box plot since data is non-normal
total_droplets=barplot_input.loc[barplot_input['origin'] == 'totals']
total_nuc=total_droplets.loc[total_droplets['datasetid'] == 'actual']
total_droplets=total_droplets.loc[total_droplets['datasetid'] == 'empty']

unique_droplets=barplot_input.loc[barplot_input['origin'] == 'unique']
unique_nuc=unique_droplets.loc[unique_droplets['datasetid'] == 'actual']
unique_droplets=unique_droplets.loc[unique_droplets['datasetid'] == 'empty']
c='#c277b1'
#altogether
fig = plt.figure(figsize =(10, 7))
# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])
# Creating plot
bp = ax.boxplot([np.log10(empty_barcodes['UMI_Count']), np.log10(nuc_barcodes['UMI_Count']),total_droplets['logged_combo'], total_nuc['logged_combo'],unique_droplets['logged_combo'], unique_nuc['logged_combo']],patch_artist=True,
            boxprops=dict(facecolor=c, color=c),medianprops=dict(color='black'))
bp=ax.set_ylim([-0.1, 3.5])
# show plot
#ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/nuc_box_whisker.svg")
plt.show()

In [ ]:
#Mann-whitney test of medians for each metric
from scipy.stats import ranksums

print('Combos')
print(ranksums(np.log10(empty_barcodes['UMI_Count']), np.log10(nuc_barcodes['UMI_Count'])))
print('Bonferroni Adjusted')
print(0.0*3)
print('Totals')
print(ranksums(total_droplets['logged_combo'], total_nuc['logged_combo']))
print('Bonferroni Adjusted')
print(0.0*3)
print('Unique')
print(ranksums(unique_droplets['logged_combo'], unique_nuc['logged_combo']))
print('Bonferroni Adjusted')
print(0.0*3)


In [ ]:
NO THRESHOLDING

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 1].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)
print(empty['GENERATED_BARCODE'].nunique())
#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 1].index, inplace=True)
print(empty_barcodes['CBC'].nunique())

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≥2

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 2].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 2].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≥3

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 3].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 3].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≥4

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 4].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 4].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≥5

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 5].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 5].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≥10

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 10].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 10].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≥100

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('n')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 100].index, inplace=True)

#Pull empty droplets for all datasets
n1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n1_empty.drop(n1_empty[n1_empty['NUM_READS'] > 10].index, inplace=True)
n1_empty['GENERATED_BARCODE'] = 'n1_' + n1_empty['GENERATED_BARCODE'].astype(str)

n2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_nuc/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
n2_empty.drop(n2_empty[n2_empty['NUM_READS'] > 10].index, inplace=True)
n2_empty['GENERATED_BARCODE'] = 'n2_' + n2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([n1_empty, n2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 100].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
CELL ONLY

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 1].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)
print(empty['GENERATED_BARCODE'].nunique())
#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 1].index, inplace=True)
print(empty_barcodes['CBC'].nunique())

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDHING AT ≤2

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 2].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 2].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≤3

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 3].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 3].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≤4

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 4].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 4].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≤5

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 5].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 5].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≤10

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 10].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 10].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
THRESHOLDING AT ≤100

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
nuc_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 
nuc_barcodes.drop(nuc_barcodes[nuc_barcodes['UMI_Count'] < 100].index, inplace=True)


#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

empty_barcodes.drop(empty_barcodes[empty_barcodes['UMI_Count'] < 100].index, inplace=True)

In [ ]:
#Prep
summed=nuc_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = nuc_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Stacked bar plot for diversity stats in empty droplets (post-thresholding)
#Number of droplet barcodes overlapping with barcodes in actual cells
real_barcodes=new_actual['barcode']
overlap=new_empty.apply(lambda row: row[new_empty['barcode'].isin(real_barcodes)])

print('Unique barcodes detected across all empty droplets:')
print(new_empty['barcode'].nunique())
print('Unique barcodes detected across all real nuc:')
print(new_actual['barcode'].nunique())
print('Unique barcodes overlapping between real nuc and empty droplets:')
print(overlap['barcode'].nunique())

#Number of barcodes unique to empty droplets
non_overlap=new_empty.apply(lambda row: row[~new_empty['barcode'].isin(real_barcodes)])
print('Unique barcodes exclusive to empty droplets:')
print(non_overlap['barcode'].nunique())

#Number of barcodes unique to actual cells
empty_barcodes=new_empty['barcode']
real_non_overlap=new_actual.apply(lambda row: row[~new_actual['barcode'].isin(empty_barcodes)])
print('Unique barcodes exclusive to real nuc:')
print(real_non_overlap['barcode'].nunique())

#Number of cells
print('Number of usable nuc')
print(new_actual['CBC'].nunique())

#Number of empty droplets
print('Number of empty droplets')
print(new_empty['CBC'].nunique())

In [ ]:
actual_barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/no_threshold_barcodes.csv',delimiter=',')
actual_barcodes.drop(actual_barcodes[actual_barcodes['UMI_Count'] < 2].index, inplace=True)

#Pull empty droplets for all datasets
s1_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s1/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s1_empty.drop(s1_empty[s1_empty['NUM_READS'] > 10].index, inplace=True)
s1_empty['GENERATED_BARCODE'] = 's1_' + s1_empty['GENERATED_BARCODE'].astype(str)

s2_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s2/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s2_empty.drop(s2_empty[s2_empty['NUM_READS'] > 10].index, inplace=True)
s2_empty['GENERATED_BARCODE'] = 's2_' + s2_empty['GENERATED_BARCODE'].astype(str)

s3_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s3/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s3_empty.drop(s3_empty[s3_empty['NUM_READS'] > 10].index, inplace=True)
s3_empty['GENERATED_BARCODE'] = 's3_' + s3_empty['GENERATED_BARCODE'].astype(str)

s4_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s4/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s4_empty.drop(s4_empty[s4_empty['NUM_READS'] > 10].index, inplace=True)
s4_empty['GENERATED_BARCODE'] = 's4_' + s4_empty['GENERATED_BARCODE'].astype(str)

s5_empty=pd.read_table('/Users/maddieurbanek/Desktop/febmarch/s5/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
s5_empty.drop(s5_empty[s5_empty['NUM_READS'] > 10].index, inplace=True)
s5_empty['GENERATED_BARCODE'] = 's5_' + s5_empty['GENERATED_BARCODE'].astype(str)

c1_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw19.1/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c1_empty.drop(c1_empty[c1_empty['NUM_READS'] > 10].index, inplace=True)
c1_empty['GENERATED_BARCODE'] = 'c1_' + c1_empty['GENERATED_BARCODE'].astype(str)

c2_empty=pd.read_table('/Users/maddieurbanek/Desktop/apr2024_nuc_vs_single/gw21/cdna_cell/barcodes/generated_barcode_read_info_table.csv',delimiter=',')
c2_empty.drop(c2_empty[c2_empty['NUM_READS'] > 10].index, inplace=True)
c2_empty['GENERATED_BARCODE'] = 'c2_' + c2_empty['GENERATED_BARCODE'].astype(str)

empty = pd.concat([s1_empty, s2_empty, s3_empty, s4_empty, s5_empty, c1_empty, c2_empty], axis=0, ignore_index=True)

#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]


In [ ]:
#Pull rabies barcodes corresponding to empty droplets
barcodes=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/all_barcodes.csv',delimiter=',')
empty_barcodes = barcodes[barcodes['CBC'].isin(empty['GENERATED_BARCODE'])]

In [ ]:
#Get all cell-matched barcodes, w/o UMI thresholding

cell_barcodes = actualbarcodes[actualbarcodes['CBC'].str.contains('c|s')] 


In [ ]:
#Prep
summed=cell_barcodes.groupby('CBC')['UMI_Count'].sum()
summed = pd.DataFrame(summed)
summed = summed.reset_index()

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']

new_actual = cell_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']

actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()

new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')


empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()

new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')


new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']


new_actual = cell_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']


In [ ]:
#Plot total transcripts distribution
print(np.log10(summed['UMI_Count'].min()))
print(np.log10(summed['UMI_Count'].max()))
print('Mean of total transcripts per cell/nuc')
print(summed['UMI_Count'].mean())
print('Median of total transcripts per cell/nuc')
print(summed['UMI_Count'].median())
print('SEM of total transcripts for cell/nuc')
print(summed['UMI_Count'].sem())

summed_empty=empty_barcodes.groupby('CBC')['UMI_Count'].sum()
summed_empty = pd.DataFrame(summed_empty)
summed_empty = summed_empty.reset_index()
print(np.log10(summed_empty['UMI_Count'].min()))
print(np.log10(summed_empty['UMI_Count'].max()))
print('Mean of total transcripts per empty droplet')
print(summed_empty['UMI_Count'].mean())
print('Median of total transcripts per empty droplet')
print(summed_empty['UMI_Count'].median())
print('SEM of total transcripts for cell/nuc')
print(summed_empty['UMI_Count'].sem())

summed.to_csv('/Users/maddieurbanek/Desktop/summed_cell.csv')
summed_empty.to_csv('/Users/maddieurbanek/Desktop/summed_empty_cell.csv')

plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist((np.log10(summed['UMI_Count'])),bins=np.arange(0, 4.88024177589548 + .01, .05),alpha=0.5,label='Real Cells')
plt.hist((np.log10(summed_empty['UMI_Count'])),bins=np.arange(0, 4.88024177589548 + .01, .05),alpha=0.3,color='green',label='Empty Droplets')

plt.xlim([-.1, 5])
plt.legend(loc='upper right')
plt.xlabel('Log10(Total Barcode Transcripts per Cell)')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_total_transcripts.svg', bbox_inches='tight', format='svg')
plt.show()

print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
from scipy.stats import ranksums

drop_dist = summed_empty['UMI_Count']
cell_dist = summed['UMI_Count']

print('Wilcoxon Ranked Sum Results:')
print(ranksums(drop_dist, cell_dist))
print(len(summed))
print(len(summed_empty))

In [ ]:
#Plot combo
print((cell_barcodes['UMI_Count'].min()))
print((cell_barcodes['UMI_Count'].max()))
print('Mean of unique barcode transcripts per cell/nuc')
print(cell_barcodes['UMI_Count'].mean())
print('Median of unique barcode transcripts per cell/nuc')
print(cell_barcodes['UMI_Count'].median())
print('SEM of total transcripts for cell/nuc')
print(cell_barcodes['UMI_Count'].sem())


print((empty_barcodes['UMI_Count'].min()))
print((empty_barcodes['UMI_Count'].max()))
print('Mean of unique barcode transcripts per empty droplet')
print(empty_barcodes['UMI_Count'].mean())
print('Median of unique barcode transcripts per empty droplet')
print(empty_barcodes['UMI_Count'].median())
print('SEM of total transcripts for cell/nuc')
print(empty_barcodes['UMI_Count'].sem())

plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist(((cell_barcodes['UMI_Count'])),bins=np.arange(1, 24169 + 100, 250),alpha=0.5,label='Real Cells')
plt.hist(((empty_barcodes['UMI_Count'])),bins=np.arange(1, 24169 + 100, 250),alpha=0.3,color='green',label='Empty Droplets')


#plt.xlim([-.1, 5])
plt.legend(loc='upper right')
plt.xlabel('Barcode Transcripts per Cell:Barcode Combination')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_combo_transcripts.svg', bbox_inches='tight', format='svg')
plt.show()

print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
from scipy.stats import ranksums

drop_dist = empty_barcodes['UMI_Count']
cell_dist = cell_barcodes['UMI_Count']

print('Wilcoxon Ranked Sum Results:')
print(ranksums(drop_dist, cell_dist))
print(len(empty_barcodes))
print(len(cell_barcodes))

In [ ]:
#Zoomed
plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist(((cell_barcodes['UMI_Count'])),bins=np.arange(1, 24169 + 100, 10),alpha=0.5,label='Real Cells')
plt.hist(((empty_barcodes['UMI_Count'])),bins=np.arange(1, 24169 + 100, 10),alpha=0.3,color='green',label='Empty Droplets')

plt.xlim([-10, 600])
plt.legend(loc='upper right')
plt.xlabel('Log10(Barcode Transcripts per Cell:Barcode Combination)')
plt.ylabel('Frequency')
plt.yscale('log')
plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_zoomed_combo_transcripts.svg', bbox_inches='tight', format='svg')
plt.show()

In [ ]:
#Calculate % of UMIs each barcode takes up per cell

new_empty = empty_barcodes.merge(summed_empty[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_empty['percent_umi']=new_empty['UMI_Count_x']/new_empty['UMI_Count_y']
print(new_empty)

new_actual = cell_barcodes.merge(summed[['CBC','UMI_Count']], on = 'CBC', how = 'left')
new_actual['percent_umi']=new_actual['UMI_Count_x']/new_actual['UMI_Count_y']
print(new_actual)

In [ ]:
actual_unique=new_actual.groupby('CBC')['barcode'].nunique()
actual_unique=pd.DataFrame(actual_unique).reset_index()
print(actual_unique)
new_actual = new_actual.merge(actual_unique[['CBC','barcode']], on = 'CBC', how = 'left')
print(new_actual)

empty_unique=new_empty.groupby('CBC')['barcode'].nunique()
empty_unique=pd.DataFrame(empty_unique).reset_index()
print(empty_unique)
new_empty = new_empty.merge(empty_unique[['CBC','barcode']], on = 'CBC', how = 'left')
print(new_empty)

new_empty['logged_total']=np.log10(new_empty['UMI_Count_y'])
new_empty['logged_unique']=np.log10(new_empty['barcode_y'])

new_actual['logged_total']=np.log10(new_actual['UMI_Count_y'])
new_actual['logged_unique']=np.log10(new_actual['barcode_y'])

combo_actual=new_actual.copy(deep=True)
combo_empty=new_empty.copy(deep=True)

new_actual=new_actual.drop_duplicates('CBC').sort_index()
print(len(new_actual))
new_empty=new_empty.drop_duplicates('CBC').sort_index()
print(len(new_empty))

new_actual.to_csv('/Users/maddieurbanek/Desktop/actual_cell.csv')
new_empty.to_csv('/Users/maddieurbanek/Desktop/actual_empty_cell.csv')

In [ ]:
#Plot unique number of barcodes per cell/droplet
print(new_actual['logged_unique'].min())
print(new_actual['logged_unique'].max())
print('Mean of unique barcodes for cell/nuc')
print(new_actual['barcode_y'].mean())
print('Median of unique barcodes for cell/nuc')
print(new_actual['barcode_y'].median())
print('SEM of unique barcodes for cell/nuc')
print(new_actual['barcode_y'].sem())

print((new_empty['logged_unique'].min()))
print(new_empty['logged_unique'].max())
print('Mean of unique barcodes for empty droplet')
print(new_empty['barcode_y'].mean())
print('Median of unique barcodes for empty droplet')
print(new_empty['barcode_y'].median())
print('SEM of unique barcodes for empty droplet')
print(new_empty['barcode_y'].sem())

plt.figure().set_figheight(5)
#Set bins to the max values printed above

plt.hist(((new_actual['logged_unique'])),bins=np.arange(0, 3.074084689028244 + .01, .05),alpha=0.5,label='Real Cells')
plt.hist(((new_empty['logged_unique'])),bins=np.arange(0, 3.074084689028244 + .01, .05),alpha=0.3,color='green',label='Empty Droplets')

plt.legend(loc='upper right')
plt.xlabel('Log10(Unique Barcodes)')
plt.ylabel('Frequency')
plt.yscale('log')
#plt.savefig('/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_unique_barcode.svg', bbox_inches='tight', format='svg')
plt.show()


print('Wilcoxon Ranked Sum to compare distributions:')

import numpy as np
from scipy.stats import ranksums

drop_dist = new_empty['logged_unique']
cell_dist = new_actual['logged_unique']

print('Wilcoxon Ranked Sum Results:')
print(ranksums(drop_dist, cell_dist))
print(len(drop_dist))
print(len(cell_dist))

In [ ]:
#Prep datasets for bar plots
new_empty['logged_combo']=np.log10(new_empty['UMI_Count_x'])
new_actual['logged_combo']=np.log10(new_actual['UMI_Count_x'])
new_actual['datasetid']='actual'
new_empty['datasetid']='empty'
barplot=pd.concat([new_actual, new_empty], axis=0)

In [ ]:
combos = barplot[['logged_combo', 'datasetid']].copy()
combos[['origin']]='combo'
print(combos)

subset_barplot=barplot.copy(deep=True)
subset_barplot=subset_barplot.drop_duplicates(subset=['CBC'])

totals = subset_barplot[['logged_total', 'datasetid']].copy()
totals.columns=['logged_combo','datasetid']
totals[['origin']]='totals'
print(totals)

uniques = subset_barplot[['logged_unique', 'datasetid']].copy()
uniques.columns=['logged_combo','datasetid']
uniques[['origin']]='unique'
print(uniques)

barplot_input=pd.concat([combos, totals, uniques], axis=0)
print(barplot_input)

In [ ]:
#Summary stats bar plot
ax=sns.barplot(data=barplot_input, x="origin", y="logged_combo", hue="datasetid",errorbar="sd",zorder=10)
#ax=sns.swarmplot(x="origin", y="logged_combo", data=barplot_input, color="0", alpha=.1)
ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_pre_stats.svg")
plt.show()

print('Welchs T-test to compare means for cell:barcode combos:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(new_actual['logged_combo'], new_empty['logged_combo'],equal_var=False))

print('Welchs T-test to compare means for total transcripts:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(new_actual['logged_total'], new_empty['logged_total'],equal_var=False))

print('Welchs T-test to compare means for unique barcodes:')
import numpy as np
from scipy import stats
print(stats.ttest_ind(new_actual['logged_unique'], new_empty['logged_unique'],equal_var=False))

In [ ]:
#Box plot since data is non-normal
total_droplets=barplot_input.loc[barplot_input['origin'] == 'totals']
total_nuc=total_droplets.loc[total_droplets['datasetid'] == 'actual']
total_droplets=total_droplets.loc[total_droplets['datasetid'] == 'empty']

unique_droplets=barplot_input.loc[barplot_input['origin'] == 'unique']
unique_nuc=unique_droplets.loc[unique_droplets['datasetid'] == 'actual']
unique_droplets=unique_droplets.loc[unique_droplets['datasetid'] == 'empty']

#altogether
c='#429ac8'
fig = plt.figure(figsize =(10, 7))
# Creating axes instance
ax = fig.add_axes([0, 0, 1, 1])
# Creating plot
bp = ax.boxplot([np.log10(empty_barcodes['UMI_Count']), np.log10(cell_barcodes['UMI_Count']),total_droplets['logged_combo'], total_nuc['logged_combo'],unique_droplets['logged_combo'], unique_nuc['logged_combo']],patch_artist=True,
            boxprops=dict(facecolor=c, color=c),medianprops=dict(color='black'))
bp=ax.set_ylim([-0.1, 5])
# show plot
ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_box_whisker.svg")
#ax.figure.savefig("/Users/maddieurbanek/Desktop/rabies_project/empty_droplet/cell_box_whisker.pdf")
plt.show()



In [ ]:
#Mann-whitney test of medians for each metric
from scipy.stats import ranksums

print('Combos')
print(ranksums(np.log10(empty_barcodes['UMI_Count']), np.log10(cell_barcodes['UMI_Count'])))
print('Bonferroni Adjusted')
print(0.0*3)
print('Totals')
print(ranksums(total_droplets['logged_combo'], total_nuc['logged_combo']))
print('Bonferroni Adjusted')
print(0.0*3)
print('Unique')
print(ranksums(unique_droplets['logged_combo'], unique_nuc['logged_combo']))
print('Bonferroni Adjusted')
print(0.0*3)


In [ ]:
FIG 8: Connectivity analysis (descriptive stats)

In [ ]:
#Stacked bar plots in connectivity fig
cell_type_counts=pd.read_table('/Users/maddieurbanek/Desktop/rabies_project/data/cell_types_per_network.csv',delimiter=',')
two_cells=(cell_type_counts[cell_type_counts['number of cells in single-starter network'] == 2])
print('two cell networks')
print(two_cells['celltype'].value_counts())
two_cells=(cell_type_counts[cell_type_counts['number of cells in single-starter network'] >2])
two_cells=(two_cells[two_cells['number of cells in single-starter network'] <11])
print('three-ten cell networks')
print(two_cells['celltype'].value_counts())
two_cells=(cell_type_counts[cell_type_counts['number of cells in single-starter network'] >10])
two_cells=(two_cells[two_cells['number of cells in single-starter network'] <21])
print('11-20 cell networks')
print(two_cells['celltype'].value_counts())
two_cells=(cell_type_counts[cell_type_counts['number of cells in single-starter network'] >20])
print('>20 cell networks')
print(two_cells['celltype'].value_counts())


In [ ]:
#plot cell type proportions per network
df = pd.DataFrame([['2', 8,7,9,4,20,145,20,15], ['3-10', 43,25,57,19,79,754,130,78], ['11-20', 27,17,35,13,54,573,72,43],
                   ['>20', 134,51,96,81,243,1610,221,105]],
                  columns=['network_size','rg', 'ol', 'ac', 'pr', 'ga','l2','l4','l5'])
# view data
print(df)
 
# plot data in stack manner of bar type
df.plot(x='network_size', kind='bar', stacked=True,
        title='')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/8_connectivity/cell_type_network_size.svg",bbox_inches='tight', format='svg')
plt.show()




#plot cell type proportions per network
df = pd.DataFrame([['2',0.035087719,0.030701754,0.039473684,0.01754386,0.087719298,0.635964912,0.087719298,0.065789474 ], ['3-10',0.03628692,0.021097046,0.048101266,0.016033755,0.066666667, 0.63628692,0.109704641,0.065822785], ['11-20',0.032374101,0.020383693,0.041966427,0.01558753,0.064748201,0.68705036,0.086330935,0.051558753 ],
                   ['>20',0.052735144,0.020070838,0.037780401,0.031877214,0.095631641,0.633608815,0.086973632,0.041322314 ]],
                  columns=['network_size','rg', 'ol', 'ac', 'pr', 'ga','l2','l4','l5'])
# view data
print(df)
 
# plot data in stack manner of bar type
df.plot(x='network_size', kind='bar', stacked=True,
        title='')
plt.draw()
plt.savefig("/Users/maddieurbanek/Desktop/rabies_project/8_connectivity/cell_type_network_size_props.svg",bbox_inches='tight', format='svg')
plt.show()
